# A la fin et pense bête

- Check la corrélation lien linkedin et nom prenom
- Limitation, scrap ne prend que les deux dernières formations (car il faut déplier sur page profil LinkedIn pour avoir la suite)

# Librairie

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import json
import glob
from urllib.parse import quote

# Trouver les liens des profils LinkedIn manquant avec PhantomBuster

## Identification des liens profil linkedin manquant

- Option 1 : 0,1 par profil trouvé soit 100$ / 1000 profil https://console.apify.com/actors/M2FMdjRVeF1HPGFcc/information/latest/readme
- Option 2 : Paiement à l"heure d'utilisation mais nécessite cookie LinkedIn https://phantombuster.com/8088789039214491/phantoms/7412035120602675/setup/step/searches-to-export?returnTo=%2F8088789039214491%2Fphantoms%2F7412035120602675%2Fconsole

In [2]:
EPSI_promotion_brut = pd.read_excel("AFINUM EPSI Tableau de promotions 24092025 pour scrapping.xlsx", sheet_name="OUTIL DE COLLECTE", engine='openpyxl', header=2) # Data de base fournit par C&D

In [3]:
# Ajout de la colonne "Lien FantomBuster" dans EPSI_promotion_brut

# Template URL
template_url = "https://www.linkedin.com/search/results/people/?keywords={keywords}&origin=GLOBAL_SEARCH_HEADER&schoolFilter=%5B%22607717%22%5D"

# Création de la colonne url_link et création d'une copie du dataframe
EPSI_promotion_brut["Lien FantomBuster avec ecole"] = EPSI_promotion_brut.apply(lambda row: template_url.format(keywords=quote(row["Nom et Prénom du titulaire"])), axis=1)
EPSI_promotion_brut.to_excel("AFINUM EPSI Tableau de promotions 24092025 pour scrapping - CODE COPIE.xlsx", index=False)

## Import des données

### Résultat du scraping de PhantomBuster en xlsx

In [4]:
# Import de tous les exports apify
file_paths = glob.glob("Export phantombuster/*.xlsx")

# Charger tous les fichiers Excel dans une liste, chaque élément étant un DataFrame
Phantombuster_exports = [pd.read_excel(f) for f in file_paths]

# Concaténer tous les DataFrames en un seul
Phantombuster_exports_total = pd.concat(Phantombuster_exports, ignore_index=True)

# Filtre des colonnes utiles
Phantombuster_exports_total = Phantombuster_exports_total[["profileUrl", "fullName", "fullName_from_input"]].drop_duplicates().reset_index(drop=True)

# Filtre des profils non trouvé
Phantombuster_exports_total = Phantombuster_exports_total[~pd.isna(Phantombuster_exports_total["profileUrl"])].reset_index(drop=True)

In [5]:
Phantombuster_exports_total # profil trouvé de la part de phantom buster avec nom prenom et filtre ecole = EPSI - ecole d'informatique truc muche

# A rajouter au fichier EPSI_promotion_brut.xlsx dans la colonne "Lien profil linkedin" les liens trouvés
EPSI_promotion_brut = EPSI_promotion_brut.merge(Phantombuster_exports_total, left_on="Lien FantomBuster avec ecole", right_on="fullName_from_input", how="left")

# Ajoute les liens trouvé dans la colonne "Lien profil linkedin" si elle est vide ou contient "Absence de Compte LinkedIn"
EPSI_promotion_brut["Lien profil linkedin"] = np.where(
    EPSI_promotion_brut["Lien profil linkedin"].isna() | 
    EPSI_promotion_brut["Lien profil linkedin"].str.strip().eq("") |
    EPSI_promotion_brut["Lien profil linkedin"].str.lower().eq("absence de compte linkedin"),
    EPSI_promotion_brut["profileUrl"],
    EPSI_promotion_brut["Lien profil linkedin"]
)

In [6]:
EPSI_promotion_brut.to_excel("AFINUM EPSI Tableau de promotions 24092025 pour scrapping - CODE COPIE.xlsx", index=False)

# TRAITEMENT DES DONNEES PROVENANT DU SCRAPING DE APIFY

## Import des données

### Résultat du scraping de APIFY en xlsx

- Le scraping a été fait avec APIFY sous ce lien : https://console.apify.com/actors/2SyF0bVxmgGr8IVCZ/input

In [7]:
# Import de tous les exports apify
file_paths = glob.glob("export apify/*.xlsx")

# Charger tous les fichiers Excel dans une liste, chaque élément étant un DataFrame
Apify_exports = [pd.read_excel(f) for f in file_paths]

# Concaténer tous les DataFrames en un seul
Apify_export_total_brut = pd.concat(Apify_exports, ignore_index=True)

### Fichier brut des élèves fournit par C&D en xlsx

In [8]:
#EPSI_promotion_brut = pd.read_excel("AFINUM EPSI Tableau de promotions 24092025 pour scrapping.xlsx", sheet_name="OUTIL DE COLLECTE", engine='openpyxl', header=2) # Data de base fournit par C&D
EPSI_promotion_brut = pd.read_excel("AFINUM EPSI Tableau de promotions 24092025 pour scrapping - CODE COPIE.xlsx", engine='openpyxl')

## Création du dataset EPSI_scraping_result : EPSI_promotion + Apify_export_total

### Préparation Apify_export_total

In [9]:
# Filtre les données pour ne garder que les profils avec un poste actuel ou une formation ce qui veut dire que le scraping a fonctionné
Apify_export_total_filtre_scraping_reussi = Apify_export_total_brut[(~Apify_export_total_brut["jobTitle"].isna()) | (~Apify_export_total_brut["educations/0/title"].isna())]

# Suppression des doublons en gardant la première occurrence
Apify_export_total_doublon = Apify_export_total_filtre_scraping_reussi.drop_duplicates(subset=["linkedinUrl"], keep="first")

In [10]:
# Sélection des colonnes d'intérêt
Apify_export_total_selection = Apify_export_total_doublon[[
                                "fullName",
                                "linkedinUrl",
                                
                            # Education
                                # Ecole
                                "educations/0/title",
                                "educations/1/title",
                                # Diplôme
                                "educations/0/subtitle",
                                "educations/1/subtitle",
                                # Année diplôme
                                "educations/0/caption", 
                                "educations/1/caption", 

                            # Expériences
                                # Poste
                                "jobTitle",
                                "experiences/0/title",
                                "experiences/1/title", 
                                "experiences/2/title", 
                                "experiences/3/title", 
                                "experiences/4/title",
                                # Date
                                "experiences/0/caption",
                                "experiences/1/caption",
                                "experiences/2/caption",
                                "experiences/3/caption",
                                "experiences/4/caption",
                                # Nom entreprise + type contrat          
                                "experiences/0/subtitle",
                                "experiences/1/subtitle",
                                "experiences/2/subtitle",
                                "experiences/3/subtitle",
                                "experiences/4/subtitle",

                                # Si plusieurs postes dans une même entreprise poste 1
                                "experiences/0/subComponents/0/title",
                                "experiences/0/subComponents/1/title",
                                "experiences/0/subComponents/2/title",
                                # Si plusieurs postes dans une même entreprise date 1
                                "experiences/0/subComponents/0/caption",
                                "experiences/0/subComponents/1/caption",
                                "experiences/0/subComponents/2/caption",
                                # Si plusieurs postes dans une même entreprise contrat 1
                                "experiences/0/subComponents/0/subtitle",
                                "experiences/0/subComponents/1/subtitle",
                                "experiences/0/subComponents/2/subtitle",

                                # Si plusieurs postes dans une même entreprise poste 2
                                "experiences/1/subComponents/0/title",
                                "experiences/1/subComponents/1/title",
                                "experiences/1/subComponents/2/title",
                                # Si plusieurs postes dans une même entreprise date 2
                                "experiences/1/subComponents/0/caption",
                                "experiences/1/subComponents/1/caption",
                                "experiences/1/subComponents/2/caption",
                                # Si plusieurs postes dans une même entreprise contrat 2
                                "experiences/1/subComponents/0/subtitle",
                                "experiences/1/subComponents/1/subtitle",
                                "experiences/1/subComponents/2/subtitle",

                                # Si plusieurs postes dans une même entreprise poste 3
                                "experiences/2/subComponents/0/title",
                                "experiences/2/subComponents/1/title",
                                "experiences/2/subComponents/2/title",
                                # Si plusieurs postes dans une même entreprise date 3
                                "experiences/2/subComponents/0/caption",
                                "experiences/2/subComponents/1/caption",
                                "experiences/2/subComponents/2/caption",
                                # Si plusieurs postes dans une même entreprise contrat 3
                                "experiences/2/subComponents/0/subtitle",
                                "experiences/2/subComponents/1/subtitle",
                                "experiences/2/subComponents/2/subtitle",

                                # Si plusieurs postes dans une même entreprise poste 4
                                "experiences/3/subComponents/0/title",
                                "experiences/3/subComponents/1/title",
                                "experiences/3/subComponents/2/title",
                                # Si plusieurs postes dans une même entreprise date 4
                                "experiences/3/subComponents/0/caption",
                                "experiences/3/subComponents/1/caption",
                                "experiences/3/subComponents/2/caption",
                                # Si plusieurs postes dans une même entreprise contrat 4
                                "experiences/3/subComponents/0/subtitle",
                                "experiences/3/subComponents/1/subtitle",
                                "experiences/3/subComponents/2/subtitle",

                                # Si plusieurs postes dans une même entreprise poste 5
                                "experiences/4/subComponents/0/title",
                                "experiences/4/subComponents/1/title",
                                "experiences/4/subComponents/2/title",
                                # Si plusieurs postes dans une même entreprise date 5
                                "experiences/4/subComponents/0/caption",
                                "experiences/4/subComponents/1/caption",
                                "experiences/4/subComponents/2/caption",
                                # Si plusieurs postes dans une même entreprise contrat 5
                                "experiences/4/subComponents/0/subtitle",
                                "experiences/4/subComponents/1/subtitle",
                                "experiences/4/subComponents/2/subtitle"

                            # Bonus                              
                            ]]

In [11]:
# Renommage des colonnes pour une meilleure lisibilité
colonnes = {
    "fullName": "Nom complet",
    "linkedinUrl": "Lien profil linkedin scraping",

# Education
    # Ecole
    "educations/0/title": "École 1",
    "educations/0/subtitle": "Diplôme 1",
    "educations/0/caption": "Date diplôme 1",
    "educations/1/title": "École 2",
    "educations/1/subtitle": "Diplôme 2",
    "educations/1/caption": "Date diplôme 2",

    # Expériences
    "jobTitle" : "Poste actuel",
    "experiences/0/title" : "Poste 1",
    "experiences/1/title" : "Poste 2",
    "experiences/2/title" : "Poste 3",
    "experiences/3/title" : "Poste 4",
    "experiences/4/title" : "Poste 5",
    
    "experiences/0/caption" : "Date poste 1",
    "experiences/1/caption" : "Date poste 2",
    "experiences/2/caption" : "Date poste 3",
    "experiences/3/caption" : "Date poste 4",
    "experiences/4/caption" : "Date poste 5",

    "experiences/0/subtitle" : "Nom entreprise + type contrat 1",
    "experiences/1/subtitle" : "Nom entreprise + type contrat 2",
    "experiences/2/subtitle" : "Nom entreprise + type contrat 3",
    "experiences/3/subtitle" : "Nom entreprise + type contrat 4",
    "experiences/4/subtitle" : "Nom entreprise + type contrat 5",

    # Si plusieurs postes dans une même entreprise poste 1
    "experiences/0/subComponents/0/title" : "Poste 1.1",
    "experiences/0/subComponents/1/title" : "Poste 1.2",
    "experiences/0/subComponents/2/title" : "Poste 1.3",
    # Si plusieurs postes dans une même entreprise date 1
    "experiences/0/subComponents/0/caption" : "Date poste 1.1",
    "experiences/0/subComponents/1/caption" : "Date poste 1.2",
    "experiences/0/subComponents/2/caption" : "Date poste 1.3",
# Si plusieurs postes dans une même entreprise contrat 1
    "experiences/0/subComponents/0/subtitle" : "Nom entreprise + type contrat 1.1",
    "experiences/0/subComponents/1/subtitle" : "Nom entreprise + type contrat 1.2",
    "experiences/0/subComponents/2/subtitle" : "Nom entreprise + type contrat 1.3",

    # Si plusieurs postes dans une même entreprise poste 2
    "experiences/1/subComponents/0/title" : "Poste 2.1",
    "experiences/1/subComponents/1/title" : "Poste 2.2",
    "experiences/1/subComponents/2/title" : "Poste 2.3",
    # Si plusieurs postes dans une même entreprise date 2
    "experiences/1/subComponents/0/caption" : "Date poste 2.1",
    "experiences/1/subComponents/1/caption" : "Date poste 2.2",
    "experiences/1/subComponents/2/caption" : "Date poste 2.3",
# Si plusieurs postes dans une même entreprise contrat 2
    "experiences/1/subComponents/0/subtitle" : "Nom entreprise + type contrat 2.1",
    "experiences/1/subComponents/1/subtitle" : "Nom entreprise + type contrat 2.2",
    "experiences/1/subComponents/2/subtitle" : "Nom entreprise + type contrat 2.3",

    # Si plusieurs postes dans une même entreprise poste 3
    "experiences/2/subComponents/0/title" : "Poste 3.1",
    "experiences/2/subComponents/1/title" : "Poste 3.2",
    "experiences/2/subComponents/2/title" : "Poste 3.3",
    # Si plusieurs postes dans une même entreprise date 3
    "experiences/2/subComponents/0/caption" : "Date poste 3.1",
    "experiences/2/subComponents/1/caption" : "Date poste 3.2",
    "experiences/2/subComponents/2/caption" : "Date poste 3.3",
    # Si plusieurs postes dans une même entreprise contrat 3
    "experiences/2/subComponents/0/subtitle" : "Nom entreprise + type contrat 3.1",
    "experiences/2/subComponents/1/subtitle" : "Nom entreprise + type contrat 3.2",
    "experiences/2/subComponents/2/subtitle" : "Nom entreprise + type contrat 3.3",

    # Si plusieurs postes dans une même entreprise poste 4
    "experiences/3/subComponents/0/title" : "Poste 4.1",
    "experiences/3/subComponents/1/title" : "Poste 4.2",
    "experiences/3/subComponents/2/title" : "Poste 4.3",
    # Si plusieurs postes dans une même entreprise date 4
    "experiences/3/subComponents/0/caption" : "Date poste 4.1",
    "experiences/3/subComponents/1/caption" : "Date poste 4.2",
    "experiences/3/subComponents/2/caption" : "Date poste 4.3",
    # Si plusieurs postes dans une même entreprise contrat 4
    "experiences/3/subComponents/0/subtitle" : "Nom entreprise + type contrat 4.1",
    "experiences/3/subComponents/1/subtitle" : "Nom entreprise + type contrat 4.2",
    "experiences/3/subComponents/2/subtitle" : "Nom entreprise + type contrat 4.3",

    # Si plusieurs postes dans une même entreprise poste 5
    "experiences/4/subComponents/0/title" : "Poste 5.1",
    "experiences/4/subComponents/1/title" : "Poste 5.2",
    "experiences/4/subComponents/2/title" : "Poste 5.3",
    # Si plusieurs postes dans une même entreprise date 5
    "experiences/4/subComponents/0/caption" : "Date poste 5.1",
    "experiences/4/subComponents/1/caption" : "Date poste 5.2",
    "experiences/4/subComponents/2/caption" : "Date poste 5.3",
    # Si plusieurs postes dans une même entreprise contrat 5
    "experiences/4/subComponents/0/subtitle" : "Nom entreprise + type contrat 5.1",
    "experiences/4/subComponents/1/subtitle" : "Nom entreprise + type contrat 5.2",
    "experiences/4/subComponents/2/subtitle" : "Nom entreprise + type contrat 5.3"
}

Apify_export_total_titre = Apify_export_total_selection[list(colonnes)].rename(columns=colonnes)

### Préparation EPSI_promotion

In [12]:
# Filtre les données pour ne garder que les profils LinkedIn contenant un lien LinkedIn
#EPSI_promotion_filtre_linkedin = EPSI_promotion_brut[(EPSI_promotion_brut["Lien profil linkedin"].str.contains("linkedin.com/in/"))==True]

# Sélection des colonnes d'intérêt
EPSI_promotion_selection = EPSI_promotion_brut[[
                                "Date de la décision d'attribution de la certification",
                                "Année d'obtention de la certification",
                                "Le cas échéant option choisie",
                                "Nom et Prénom du titulaire",
                                "Dénomination de l'organisme ayant assuré la formation (sauf candidat libre et reconnaissance des acquis)",
                                "Lien profil linkedin",
                                "Modalités pédagogiques (présentiel, distanciel, hybride, AFEST)",
                                "Intitulé de la formation ayant préparé au projet ou à la certification (sauf candidat libre, reconnaissance des acquis et VAE)",
                                "Voie d'accès (formation initiale hors altenance, formation continue hors alternance, contrat d'apprentissage, contrat de professionnalisation, reconnaissance des acquis, VAE, candidat individuel, autre, non répondant)"                    
                            ]]

### Création du dataset final : EPSI_scraping_result

In [13]:
EPSI_scraping_result = pd.merge(EPSI_promotion_selection, Apify_export_total_titre, how="left", left_on="Lien profil linkedin", right_on="Lien profil linkedin scraping")

## Règle métier pour définition des colonnes

### Constantes

In [14]:
# Date servant de comparaison pour la partie poste actuel / long terme 
DATE_LONG_TERME_COMPARAISON = pd.to_datetime("30/09/2025", format="%d/%m/%Y") # Date choisi pour l'analyse long terme
DATE_LONG_TERME_LIMIT_DIPLOME = DATE_LONG_TERME_COMPARAISON - pd.DateOffset(years=5) # Date servant de comparaison, si le diplôme est toujours en cours à aujourd'hui et que le début du diplôme est postérieur au 01/07/2022 on dit que profil non à jour car pas possible + 5 ans


# Mots clés pour les règles métier
INDEPENDANT_KEYWORDS = [
    "indépendant", "independant", "freelance", "entrepreneur", "auto-entrepreneur",
    "self-employed", "fondateur", "founder", "président", "gerant", "gérant", 
    "consultant indépendant", "en mission", "Chef d'entreprise"
]

PAUSE_KEYWORDS = [
    "pause pro", "pause professionnelle", "travel", "voyage", "world tour",
    "tour du monde", "sabbatical", "année sabbatique", "gap year",
    "career break", "année de césure", "personal goal pursuit"
]

EDUCATION_KEYWORDS = [
    "alternance", "apprentissage", "apprenticeship", "formation", "étudiant", "etudiant",
    "alternant", "mastère", "master", "bachelor", "licence", "stage", "stagiaire",
    "élève", "eleve", "ecole", "school"
]

STAGE_KEYWORDS = ["stage", "stagiaire"]

ALTERNANCE_KEYWORDS = ["alternance", "apprentissage", "apprenticeship", "alternant"]

RECHERCHE_EMPLOI = [
    "en recherche d'emploi", "recherche d'une nouvelle mission",
    "sans emploi", "en transition", "looking for a job", "available"
]


# Mots clés pour les fonctions de déductions
CADRE_KEYWORDS = [
    "directeur", "directrice", "gérant", "gerant", "fondateur", "founder", 
    "président", "entrepreneur", "freelance", "independant", "indépendant",
    "manager", "responsable", "lead",
    "chargé de projet", "chargé de mission", "ingénieur", "ingenieur", "engineer",
    "consultant", "superviseur", "coordinateur", "product owner", "head of", "cto", "ceo",
    "Chef d'entreprise", "chef", "chief", "senior", "sénior"
]

NON_CADRE_KEYWORDS = [
    "technicien", "assistant", "assistante", "employé",
    "ouvrier", "agent", "opérateur", "opératrice"
]

CDI_KEYWORDS = ["cdi", "permanent", "contrat à durée indéterminée", "directeur", "directrice", "gérant", "gerant", "président"]

CDD_KEYWORDS = ["cdd", "contract", "temporary", "contrat à durée déterminée", "intérim", "interim"]

INDEPENDANT_CONTRAT_KEYWORDS = ["freelance", "indépendant", "indépendant", "auto-entrepreneur", "consultant indépendant", "self-employed", "Chef d'entreprise"]


# Niveau de qualifications du dernier diplôme obtenu
NIVEAU_4_KEYWORDS = ["niveau 4", "baccalauréat", ]
NIVEAU_5_KEYWORDS = ["bts", "brevet de technicien supérieur", "bac+2", "bac +2", "niveau 5", "cpge", "rncp ii"]
NIVEAU_6_KEYWORDS = ["dut", "bachelor", "license", "niveau 6", "m1", "master 1", "diplôme universitaire de technologie", "b3"]
NIVEAU_7_KEYWORDS = ["ingénieur", "ingenieur", "master", "mastère", "7", "m2"]

### Fonctions de déduction

In [15]:
def extract_contrat(x):
    """Vérifie la présence de mots clés dans le champ contrat pour déterminer le type de contrat."""

    if not isinstance(x, str):  # Si NaN ou autre type → transformer en string vide
        x = ""
    x = x.lower()

    if any(word in x for word in CDI_KEYWORDS):
        return "CDI"

    if any(word in x for word in CDD_KEYWORDS):
        return "CDD"

    if any(word in x for word in INDEPENDANT_CONTRAT_KEYWORDS):
        return "Indépendant"
    
    return ""

In [16]:
def extract_statut_cadre(x):
    """Vérifie la présence de mots clés dans le champ statut pour déterminer le type de statut."""

    if not isinstance(x, str):  # Si NaN ou autre type → transformer en string vide
        x = ""
    x = x.lower()

    if any(word in x for word in CADRE_KEYWORDS):
        return "Cadre"

    if any(word in x for word in NON_CADRE_KEYWORDS):
        return "Non cadre"

    return ""

In [17]:
def extract_quotite(x):
    """Vérifie la présence de mots clés dans le champ quotité."""

    if not isinstance(x, str):  # Si NaN ou autre type → transformer en string vide
        x = ""
    x = x.lower()

    if "full-time" in x.lower():
        return "100"
    else:
        return ""

In [18]:
def extract_niveau_qualification(x):
    """Vérifie la présence de mots clés dans le champ contrat pour déterminer le type de contrat."""

    if not isinstance(x, str):
        x = ""
    x = x.lower()

    if any(word in x for word in NIVEAU_7_KEYWORDS):
        return "Niveau 7"
    
    if any(word in x for word in NIVEAU_6_KEYWORDS):
        return "Niveau 6"
    
    if any(word in x for word in NIVEAU_5_KEYWORDS):
        return "Niveau 5"
    
    if any(word in x for word in NIVEAU_4_KEYWORDS):
        return "Niveau 4"
    
    return ""

### Gestion des colonnes date

In [19]:
# Transforme les dates en deux colonnes date début et date fin
"""
    Transforme les dates en deux colonnes date début et date fin.
    Utilise le format mm-aaaa pour les dates en sortie.
    Les dates concernent les colonnes "Date poste X" et "Date diplôme X".
    """


# Liste des colonnes à traiter
cols = ["Date poste 1", "Date poste 2", "Date poste 3", "Date poste 4", "Date poste 5", "Date poste 1.1", "Date poste 1.2", "Date poste 1.3", "Date poste 2.1", "Date poste 2.2",
        "Date poste 2.3", "Date poste 3.1", "Date poste 3.2", "Date poste 3.3", "Date poste 4.1", "Date poste 4.2", "Date poste 4.3", "Date poste 5.1", "Date poste 5.2", "Date poste 5.3",
        "Date diplôme 1", "Date diplôme 2"
        ]

for col in cols:
    EPSI_scraping_result[col] = EPSI_scraping_result[col].astype(str)
    
    # Extraction via regex (mois écrit en lettres + année ou Present)
    EPSI_scraping_result[[f"{col}_Début", f"{col}_Fin"]] = EPSI_scraping_result[col].str.extract(
        r"([A-Za-z]{3} \d{4}|\d{4}) - ([A-Za-z]{3} \d{4}|\d{4}|Present)"
    )
    
    # Conversion en mm-aaaa (sauf "Present", gestion YYYY)
    def convert_date(x):
        if x == "Present":
            # Remplace "Present" par le mois prochain
            return (pd.to_datetime("today") + pd.DateOffset(months=1)).strftime("%m-%Y")
        elif pd.notna(x):
            try:
                # Essaye de convertir normalement
                dt = pd.to_datetime(x, errors='coerce')
                if pd.isna(dt):
                    # Cas YYYY seul → ajouter juin
                    dt = pd.to_datetime(f"01-07-{x}", format="%d-%m-%Y", errors='coerce')
                return dt.strftime("%m-%Y")
            except:
                return x
        else:
            return x
    
    for new_col in [f"{col}_Début", f"{col}_Fin"]:
        EPSI_scraping_result[new_col] = EPSI_scraping_result[new_col].apply(convert_date)
    
    # Suppression de la colonne originale
    EPSI_scraping_result.drop(columns=[col], inplace=True)


# Convertir les colonnes de string à datetime ATTENTION si present ça met mois en cours + 1 mois
date_cols = [col for col in EPSI_scraping_result.columns if ("Date poste" in col or "Date diplôme") and ("_Début" in col or "_Fin" in col)]
for col in date_cols:
    EPSI_scraping_result[col] = pd.to_datetime(EPSI_scraping_result[col], format="%m-%Y", errors='coerce')

### Situation : Long terme / Actuel

In [20]:
# Application des règles métier pour déterminer le poste long terme

def determine_long_terme_poste(row):
    """
    Détermination du poste actuel selon les règles métier :

    1. Cherche le poste actif, soit "Poste 1" soit "Poste 1.1" (si plusieurs postes dans la même entreprise).
    2. Si la date de fin du poste est antérieure à la date de scraping => poste non valide on met "Inactif / Recherche d'emploi".
    3. Si le contrat ou l’intitulé du poste contient 'alternance' ou 'stagiaire' :
        - On prend le nom de la formation en cours,
        - Mais uniquement si la date de fin de formation est postérieure à la date de scraping.
        
    ------------------------------------    
        - Si contrat alternance -3ans  on met "En formation"
        - Si contrat stage -6mois  on met "En formation"
        - Sinon "Inactif / Recherche d'emploi"
    ------------------------------------
    
    4. Si poste est toujours vide on regarde si formation en cours de moins de 5 ans, si oui on met diplôme 1

    5. Si l’intitulé du poste contient un mot clef lié à une pause professionnelle (ex: "pause pro", "travel") on met "Inactif".
    """


    row["Long terme - Situation"] = ""

    # Règle 1 : Non répondant = Pas de scraping réaliser sur ces personnes
    if (pd.isna(row["Poste 1"]) and pd.isna(row["École 1"]) and pd.isna(row["Diplôme 1"])) or pd.isna(row["Lien profil linkedin"]):
        return ["Non répondant", "", ""]


    # Règle 2 : on cherche le poste actif
    if pd.isna(row["Poste 1.1"]) or row["Poste 1.1"] == "":
        poste = row["Poste 1"]
        poste_trouver = "1"
    else:
        poste = row["Poste 1.1"]
        poste_trouver = "1.1"


    # Règle 3 : exclusion si date antérieur au scraping
    if row["Date poste " + poste_trouver + "_Fin"] < DATE_LONG_TERME_COMPARAISON:
        row["Poste actuel"] = "Inactif / Recherche d'emploi"
    else:
        row["Poste actuel"] = poste


    # Règle 4 : cas alternance / stage
    contrat_entreprise = (str(row["Nom entreprise + type contrat 1"]) + " " + str(row["Nom entreprise + type contrat 1.1"]) + " " + str(row["Poste 1"])).lower()
    poste = str(row["Poste actuel"]).lower()

        # Si le contrat ou le poste contient 'alternance' ou 'stagiaire'
    if any(word in contrat_entreprise for word in EDUCATION_KEYWORDS) or any(word in poste for word in EDUCATION_KEYWORDS):
        if row["Date diplôme 1_Fin"] > DATE_LONG_TERME_COMPARAISON and row["Date diplôme 1_Début"] > DATE_LONG_TERME_LIMIT_DIPLOME :
            # Si alternance -3ans
            if (row["Date poste " + poste_trouver + "_Fin"] - row["Date poste " + poste_trouver + "_Début"]) <= pd.Timedelta(days=365*3) and (any(word in contrat_entreprise for word in ALTERNANCE_KEYWORDS) or any(word in poste for word in ALTERNANCE_KEYWORDS)):
                row["Poste actuel"] = row["Diplôme 1"]
                if pd.isna(row["Poste actuel"]) or row["Poste actuel"] == "":
                    row["Poste actuel"] = "Diplôme non renseigné"
                row["Long terme - Situation"] = "ecole"
            # Si stage -6mois
            elif (row["Date poste " + poste_trouver + "_Fin"] - row["Date poste " + poste_trouver + "_Début"]) <= pd.Timedelta(days=30*3) and (any(word in contrat_entreprise for word in STAGE_KEYWORDS) or any(word in poste for word in STAGE_KEYWORDS)):
                row["Poste actuel"] = row["Diplôme 1"]
                if pd.isna(row["Poste actuel"]) or row["Poste actuel"] == "":
                    row["Poste actuel"] = "Diplôme non renseigné"
                row["Long terme - Situation"] = "ecole"
            else:
                row["Poste actuel"] = "Inactif / Recherche d'emploi"
        else:
            row["Poste actuel"] = "Inactif / Recherche d'emploi"


    # Règle 5 : Si poste est toujours vide on regarde si formation en cours de moins de 5 ans, si oui on met diplôme 1
    if pd.isna(row["Poste actuel"]) or row["Poste actuel"] == "":
        if row["Date diplôme 1_Fin"] > DATE_LONG_TERME_COMPARAISON and row["Date diplôme 1_Début"] > DATE_LONG_TERME_LIMIT_DIPLOME :
            row["Poste actuel"] = row["Diplôme 1"]
            row["Long terme - Situation"] = "ecole"
        else:
            row["Poste actuel"] = "Inactif / Recherche d'emploi"

    # Règle 6 : cas pause professionnelle
    if any(word in poste for word in PAUSE_KEYWORDS) or any(word in contrat_entreprise for word in PAUSE_KEYWORDS):
        row["Poste actuel"] = "Inactif"
        row["Long terme - Situation"] = "pause"


    # Règle 7 : cas indépendant
    if any(word in poste for word in INDEPENDANT_KEYWORDS) or any(word in contrat_entreprise for word in INDEPENDANT_KEYWORDS):
        row["Long terme - Situation"] = "independant"
    

    # Règle 8 : En recherche d'emploi
    if any(word in poste for word in RECHERCHE_EMPLOI) or any(word in contrat_entreprise for word in RECHERCHE_EMPLOI):
        row["Long terme - Situation"] = "recherche emploi"
        row["Poste actuel"] = "En recherche d'emploi"


    return [row["Poste actuel"], row["Long terme - Situation"], poste_trouver]

In [21]:
# Application des règles métier pour déterminer l'entreprise long terme

def extract_entreprise(x):
    """Nettoie le champ entreprise pour enlever la partie après le séparateur '·'."""
    
    if isinstance(x, str) and "·" in x:
        return x.split("·")[0].strip()
    return x if isinstance(x, str) else ""


def determine_long_terme_entreprise(row):
    """
    Détermination de l'entreprise actuelle selon les règles métier :

    1. Si "Long terme - Poste" est vide, entreprise est vide.
    2. Si "Long terme - Poste" est "Inactif" alors entreprise est vide.
    3. Sinon on cherche "Nom entreprise + type contrat" associée au poste actuel et on le nettoie :
        - Si "Poste 1.1" et non vide, on prend "Poste 1"(Ici que se trouve le nom entreprise seul)
        - Sinon on prend "Nom entreprise + type contrat 1"
    4. Si l'entreprise ou le contrat ou le poste actuel contient des mots clés (ex. 'freelance', 'indépendant', 'self-employed'), on met "Entrepreneur".
    5. Si "l'entreprise ou le contrat contient des mots clés (ex.'alternance' ou 'stagiaire'), on met "École 1"
    6. Si entreprise est toujours vide on regarde si formation en cours de moins de 5 ans, si oui on met école 1
    """

    # Règle 1 : "Long terme - poste actuel" est vide
    if pd.isna(row["Long terme - Poste"]) or row["Long terme - Poste"] == "" or "non répondant" in row["Long terme - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "Long terme - poste actuel" contient "inactif"
    if "inactif" in row["Long terme - Poste"].lower():
        return ""
    

    # Règle 3 : "Poste 1" ou "Poste 1.1" pour choisir la bonne colonne "Nom entreprise + type contrat"
    if pd.isna(row["Poste 1.1"]) or row["Poste 1.1"] == "":
        entreprise = extract_entreprise(row["Nom entreprise + type contrat 1"])
    else:
        entreprise = row["Poste 1"]


    #Règle 4 : cas freelance / indépendant, Si l'entreprise ou le contrat contient des mots clés liés aux indépendants alors "Entrepreneur"
    if pd.notna(row["Long terme - Situation"]) and "independant" in row["Long terme - Situation"].lower():
        entreprise = "Entrepreneur"
      

    # Règle 5 : cas alternance / stage, Si l'entreprise ou le contrat ou le poste contient 'alternance' ou 'stagiaire' alors "École 1"
    if pd.notna(row["Long terme - Situation"]) and "ecole" in row["Long terme - Situation"].lower():
        entreprise = row["École 1"]
        if pd.isna(entreprise) or entreprise == "":
            entreprise = "École non renseigné"


    # Règle 6 : cas recherche emploi
    if pd.notna(row["Long terme - Situation"]) and "recherche emploi" in row["Long terme - Situation"].lower():
        return ""
    
    return entreprise

In [22]:
# Application des règles métier pour déterminer le contrat long terme  
def extract_single_value(x):
    """Extrait la première valeur se trouvant dans un liste ou transforme ['1.1'] en 1.1
    """
    if isinstance(x, (list, tuple, np.ndarray)):
        return x[0] if len(x) > 0 else None
    if isinstance(x, str):
        return x.strip("[]'\" ")
    return x


def determine_long_terme_contrat(row):
    """
    Détermination du contrat actuel selon les règles métier :

    1. Si "Long terme - Poste" est vide, contrat est "Non répondant".
    2. Si "Long terme - Poste" est "Inactif / Recherche d'emploi" alors contrat est "".
    3. Vérifie la présence de mots clés dans le champ contrat pour déterminer le type de contrat
    4. Si l'entreprise ou le contrat ou le poste actuel contient des mots clés (ex. 'freelance', 'indépendant', 'self-employed', 'fondateur'...), => "Indépendant".
    5. Si l'entreprise ou le contrat ou le poste actuel contient des mots clés (ex. 'alternance', 'stagiaire'...), => ""
    6. Si le contrat est un CDD de plus de 18 mois on le transforme en CDI
    """

    # Règle 1 : "Long terme - poste actuel" est vide
    if pd.isna(row["Long terme - Poste"]) or row["Long terme - Poste"] == "" or "non répondant" in row["Long terme - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "Long terme - poste actuel" contient "inactif"
    if "inactif" in row["Long terme - Poste"].lower():
        return ""


    # Règle 3 : "Poste 1" ou "Poste 1.1" pour choisir la bonne colonne "Nom entreprise + type contrat"
    contrat = extract_contrat(
        str(row.get("Nom entreprise + type contrat 1", "")) + " " +
        str(row.get("Nom entreprise + type contrat 1.1", "")) + " " +
        str(row.get("Poste 1", "")))


    #Règle 4 : cas freelance / indépendant, Si l'entreprise ou le contrat contient des mots clés liés aux indépendants alors "CDI"
    if pd.notna(row["Long terme - Situation"]) and "independant" in row["Long terme - Situation"].lower():
        contrat = "CDI"

    
    # Règle 5 : cas alternance / stage, Si l'entreprise ou le contrat ou le poste contient 'alternance' ou 'stagiaire' alors ""
    if pd.notna(row["Long terme - Situation"]) and "ecole" in row["Long terme - Situation"].lower():
        return ""
    

    numero_poste = extract_single_value(row["Long terme - Poste numéro"])


    # Règle 6 : Si le contrat est un CDD de plus de 18 mois on le transforme en CDI
    if contrat == "CDD":
        if numero_poste is not None:
            date_debut = row[f"Date poste {numero_poste}_Début"] if pd.notna(row[f"Date poste {numero_poste}_Début"]) else None
            date_fin = row[f"Date poste {numero_poste}_Fin"] if pd.notna(row[f"Date poste {numero_poste}_Fin"]) else None
            if date_debut and date_fin and (date_fin - date_debut) > pd.Timedelta(days=18*30):        
                contrat = "CDI"
            else:
                contrat = "CDD"


    # Règle 6 : Si le contrat est vide et que le pose a commencé il y plus de 12 mois on le transforme en CDI sinon CDD
    if contrat == "":
        if numero_poste is not None:
            date_debut = row[f"Date poste {numero_poste}_Début"] if pd.notna(row[f"Date poste {numero_poste}_Début"]) else None
            date_fin = row[f"Date poste {numero_poste}_Fin"] if pd.notna(row[f"Date poste {numero_poste}_Fin"]) else None
            if date_debut and date_fin and (date_fin - date_debut) > pd.Timedelta(days=12*30):        
                contrat = "CDI"
            else:
                contrat = "CDD"
            

    # Règle 7 : cas recherche emploi
    if pd.notna(row["Long terme - Situation"]) and "recherche emploi" in row["Long terme - Situation"].lower():
        return ""


    # Règle 8 : Si plusieurs poste dans la même entreprise alors CDI
    if numero_poste is not None and "." in numero_poste:
        contrat = "CDI"


    return contrat

In [23]:
# Application des règles métier pour déterminer le statut cadre / non-cadre long terme

def determine_long_terme_statut(row):
    """
    Détermination du statut cadre / non-cadre actuel selon les règles métier :

    1. Si "Long terme - Poste" est vide, statut est "Non répondant".
    2. Si "Long terme - Poste" est "Inactif / Recherche d'emploi" alors statut est "".
    3. Règle 3 : cas alternance / stage
    4. Vérifie la présence de mots clés dans le champ "Long terme - Poste" et "Long terme - Type de contrat"  pour déterminer le type de statut
    """

    # Règle 1 : "Long terme - poste actuel" est vide
    if pd.isna(row["Long terme - Poste"]) or row["Long terme - Poste"] == "" or "non répondant" in row["Long terme - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "Long terme - poste actuel" contient "inactif"
    if "inactif" in row["Long terme - Poste"].lower():
        return ""


    # Règle 4 : Vérifie la présence de mots clés dans le champ "Long terme - Poste"
    statut = extract_statut_cadre(str(row["Long terme - Poste"]) + " " + str(row.get("Nom entreprise + type contrat 1", "")) + " " + str(row.get("Nom entreprise + type contrat 1.1", "")))


    # Règle 5 : Indépendant
    if pd.notna(row["Long terme - Situation"]) and "independant" in row["Long terme - Situation"].lower():
        return "Cadre"


    # Règle 3 : cas alternance / stage
    if pd.notna(row["Long terme - Situation"]) and "ecole" in row["Long terme - Situation"].lower():
        return ""


    # Règle 6 : cas recherche emploi
    if pd.notna(row["Long terme - Situation"]) and "recherche emploi" in row["Long terme - Situation"].lower():
        return ""

    return statut

In [24]:
# Application des règles métier pour déterminer la quotité de travail long terme
def cherche_poste_actif_long_terme(row, max_poste=5, max_suffix=3): # 
    """"
    Cherche le ou les postes actifs à une date donnée (date du scraping).
    Retourne une liste des postes actifs ou None si aucun poste trouvé. (ex. ['Date poste 2.1'])

    arguments:
        max_poste = 5 car il y a maximum 5 entreprises différentes récupérées par élèves
        max_suffix = 3 car il y a maximum 3 postes différents dans une même entreprise
    """


    date = DATE_LONG_TERME_COMPARAISON
    postes = []
    
    for i in range(1, max_poste + 1):  # 1 à 5
        for f in range(0, max_suffix + 1):  # 0 à 3 (0 = pas de suffixe)
            # Construire le nom de colonne
            if f == 0:
                col_debut = f"Date poste {i}_Début"
                col_fin = f"Date poste {i}_Fin"
            else:
                col_debut = f"Date poste {i}.{f}_Début"
                col_fin = f"Date poste {i}.{f}_Fin"
            debut = row.get(col_debut)
            fin = row.get(col_fin)
            #Si fin est "Present", on met la date du jour + 1 mois
            if isinstance(fin, str) and fin == "Present":
                fin = pd.Timestamp.today() + pd.DateOffset(months=1)
            # Vérifier si date cible est dans l’intervalle
            if pd.notna(debut) and pd.notna(fin) and pd.notna(date):
                if debut <= date <= fin:
                    postes.append(f"{col_debut.replace('_Début','')}")
    return postes if postes else None


def determine_long_terme_quotite(row):
    """
    Détermination de la quotité de travail actuel selon les règles métier :

    1. Si "Long terme - Poste" est vide, quotité est "".
    2. Si "Long terme - Poste" est "Inactif" alors quotité est "".
    3. Si "Nom entreprise + type contrat" est auto entrepreneur / freelance alors quotité est 100
    4. Vérifie la présence de mots clés dans le champ "Nom entreprise + type contrat 1" et "Nom entreprise + type contrat 1.1" pour déterminer la quotité
    5. Si considéré comme étudiant alors ""
    6. Si il y a qu'un seul poste actif alors on met 100
    """

    # Règle 1 : "Long terme - poste actuel" est vide
    if pd.isna(row["Long terme - Poste"]) or row["Long terme - Poste"] == "" or "non répondant" in row["Long terme - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "Long terme - poste actuel" contient "inactif"
    if "inactif" in row["Long terme - Poste"].lower():
        return ""
    

    # Règle 3 : cas freelance / indépendant, Si l'entreprise ou le contrat contient des mots clés liés aux indépendants alors quotité = 100
    if pd.notna(row["Long terme - Situation"]) and "independant" in row["Long terme - Situation"].lower():
        return "100"


    # Règle 4 : Vérifie la présence de mots clés dans le champ "Nom entreprise + type contrat" si quotité non trouvée
    quotite = extract_quotite(str(row.get("Nom entreprise + type contrat 1", "")) + " " + str(row.get("Nom entreprise + type contrat 1.1", "")))


    # Règle 6 : Si il y a qu'un seul poste actif alors on met 100
    if quotite == "":
        resultat = cherche_poste_actif_long_terme(row)
        nb_elements = len(resultat) if isinstance(resultat, list) else 1
        if nb_elements == 1:
            quotite = "100"
        elif nb_elements > 1:
            quotite = "Plusieurs postes en même temps"
        else:
            quotite = None
        

    # Règle 5 : Si considéré comme étudiant alors ""
    if pd.notna(row["Long terme - Situation"]) and "ecole" in row["Long terme - Situation"].lower():
        return ""


    # Règle 6 : cas recherche emploi
    if pd.notna(row["Long terme - Situation"]) and "recherche emploi" in row["Long terme - Situation"].lower():
        return ""

    return quotite

In [25]:
# Application des règles métier pour déterminer la situation après la certification long terme

def determine_long_terme_situation_apres_certification(row):
    """
    Détermination de la situation après la certification selon les règles métier :

    1. Si "Long terme - Poste" est vide, situation est "Non répondant".
    2. Si "Long terme - Poste" est "Inactif / Recherche d'emploi" alors situation est "Inactif / Recherche d'emploi".
    3. Si "Long terme - Poste" est "Inactif" alors situation est "Inactif".
    4. Si le poste ou contrat ou entreprise contient des mots clés liés à une pause professionnelle (ex: "pause pro", "travel") on met "Inactif".
    5. Si considérer comme étudiant alors on met "En formation y compris alternance".
    6. Sinon on met "Actif occupé hors alternance".
    """

    # Règle 1 : "Long terme - poste actuel" est vide
    if pd.isna(row["Long terme - Poste"]) or row["Long terme - Poste"] == "" or "non répondant" in row["Long terme - Poste"].lower():
        return "Non répondant"


    # Règle 2 : Si "Long terme - Poste" est "Inactif / Recherche d'emploi" alors situation est "Inactif / Recherche d'emploi".
    if row["Long terme - Poste"] == "Inactif / Recherche d'emploi":
        return "Inactif / Recherche d'emploi"


    # Règle 3 : Si "Long terme - Poste" est "Inactif" alors situation est "Inactif".
    if row["Long terme - Poste"] == "Inactif":
        return "Inactif"
    

    # Règle 4 : Si considéré comme pause alors ""
    if pd.notna(row["Long terme - Situation"]) and "pause" in row["Long terme - Situation"].lower():
        return "Inactif"


    # Règle 5 : cas alternance / stage
    if pd.notna(row["Long terme - Situation"]) and "ecole" in row["Long terme - Situation"].lower():
        return "En formation y compris alternance"


    # Règle 6 : cas recherche emploi
    if pd.notna(row["Long terme - Situation"]) and "recherche emploi" in row["Long terme - Situation"].lower():
        return "En recherche d'emploi"
    

    # Règle 7 : sinon
    return "Actif occupé hors alternance"

In [26]:
# Application des règles métier pour déterminer le nombre d'années après l'obtention de la certification

def determine_long_terme_annees_apres_certification(row):
    """
    Détermination du nombre d'années après l'obtention de la certification selon les règles métier :

    1. Calcule le nombre d'années entre l'année d'obtention et l'année en cours.
    """

    
    # Règle 1 : calcul du nombre d'années entre l'année d'obtention et l'année en cours
    try:
        annee_obtention = int(row["Année d'obtention de la certification"])
        annee_actuelle = datetime.today().year
        return annee_actuelle - annee_obtention
    except ValueError:
        return ""

### Situation : 6 mois

In [27]:
# Application des règles métier pour déterminer le poste 6 mois

def cherche_poste_actif_6_mois(row, max_poste=5, max_suffix=3): # 
    """"
    Cherche le ou les postes actifs à une date donnée (6 mois après la date de jury).
    Retourne une liste des postes actifs ou None si aucun poste trouvé. (ex. ['Date poste 2.1'])

    arguments:
        max_poste = 5 car il y a maximum 5 entreprises différentes récupérées par élèves
        max_suffix = 3 car il y a maximum 3 postes différents dans une même entreprise
    """


    date = row["6 mois - Date de l'enquête dans les 6 mois - datetime"]
    postes = []
    
    for i in range(1, max_poste + 1):  # 1 à 5
        for f in range(0, max_suffix + 1):  # 0 à 3 (0 = pas de suffixe)
            # Construire le nom de colonne
            if f == 0:
                col_debut = f"Date poste {i}_Début"
                col_fin = f"Date poste {i}_Fin"
            else:
                col_debut = f"Date poste {i}.{f}_Début"
                col_fin = f"Date poste {i}.{f}_Fin"
            debut = row.get(col_debut)
            fin = row.get(col_fin)
            #Si fin est "Present", on met la date du jour + 1 mois
            if isinstance(fin, str) and fin == "Present":
                fin = pd.Timestamp.today() + pd.DateOffset(months=1)
            # Vérifier si date cible est dans l’intervalle
            if pd.notna(debut) and pd.notna(fin) and pd.notna(date):
                if debut <= date <= fin:
                    postes.append(f"{col_debut.replace('_Début','')}")
    return postes if postes else None


def remplacer_colonne_poste_par_poste(row):
    """
    Remplace le nom de la colonne du poste par le nom du poste lui-même.
    """
    postes = row["6 mois - Poste colonne"]
    if not postes:
        return None
    # On prend la première colonne trouvée (ex. 'Date poste 2.1')
    col_base = postes[0]
    # Retourne la valeur de début sur la même ligne
    return row.get(col_base)


def remplacer_colonne_diplome_par_diplome(row):
    """
    Remplace le nom de la colonne du diplôme par le nom du diplôme lui-même.
    """
    diplomes = row["6 mois - Diplôme numero"]
    if not diplomes:
        return None
    # On prend la première colonne trouvée (ex. 'Date poste 2.1')
    col_base = diplomes[0]
    # Retourne la valeur de début sur la même ligne
    return row.get(col_base)


def trouver_contrat_entreprise_associe(row):
    """
    Trouver la colonne "Nom entreprise + type contrat X" associée au poste trouvé
    """
    poste_numero = row["6 mois - Poste numero"]

    # Si c’est une liste ou un array, on prend le premier élément
    if isinstance(poste_numero, (list, pd.Series, np.ndarray)) and len(poste_numero) > 0:
        poste_key = poste_numero[0]
    # Si c’est une chaîne, on prend la chaîne entière
    elif isinstance(poste_numero, str) and len(poste_numero) > 0:
        poste_key = poste_numero
    else:
        poste_key = None

    if poste_key is None:
        return ""  # retourne chaîne vide si aucun poste

    # Poste_key_2 par défaut = poste_key (sécurisation)
    poste_key_2 = poste_key
    # Si poste_key contient un point, on prend seulement la partie avant
    if '.' in poste_key:
        poste_key_2 = poste_key.split('.')[0]

    # Retourne la concaténation sécurisée
    return (
        str(row.get("Poste " + poste_key, "")).lower() + " " +
        str(row.get("Nom entreprise + type contrat " + poste_key, "")).lower() + " " +
        str(row.get("Nom entreprise + type contrat " + poste_key_2, "")).lower()
    )


def cherche_diplome_actif_6_mois(row, max_diplome=3):
    """"
    Cherche le diplôme actifs à une date donnée (6 mois après la date de jury).

    arguments:
        max_diplome = 3 car on remonte 3 diplômes en arrière
    """


    date = row["6 mois - Date de l'enquête dans les 6 mois - datetime"]
    postes = []
    
    for i in range(1, max_diplome + 1):  # 1 à 3
        # Construire le nom de colonne
        col_debut = f"Date diplôme {i}_Début"
        col_fin = f"Date diplôme {i}_Fin"
        debut = row.get(col_debut)
        fin = row.get(col_fin)
        #Si fin est "Present", on met la date du jour + 1 mois
        if isinstance(fin, str) and fin == "Present":
            fin = pd.Timestamp.today() + pd.DateOffset(months=1)
        # Vérifier si date cible est dans l’intervalle
        if pd.notna(debut) and pd.notna(fin) and pd.notna(date):
            if debut <= date <= fin:
                postes.append(f"{col_debut.replace('_Début','')}")
    return postes if postes else None


def determine_6_mois_poste(row):
    """
    Détermination du poste à 6 mois selon les règles métier :

    1. On cherche le poste qu'occupait la personne à 6mois après la date de jury. On compare les débuts de mois par exemple si date du jury + 6mois = 15/03/2024 on prend 01/03/2024.
    2. Si le contrat ou l’intitulé du poste contient 'alternance' ou 'stagiaire' :
        - On prend le nom de la formation actif à 6 mois,
        - Mais uniquement si la date de fin de formation est postérieure à la date de jury.
        - Si contrat alternance -3ans  on met "En formation"
        - Si contrat stage -6mois  on met "En formation" sinon "Non répondant"
    3. Si l’intitulé du poste contient un mot clef lié à une pause professionnelle (ex: "pause pro", "travel") on met "Inactif".
    4. Si poste est toujours vide on regarde si formation en cours qui a commencé moins de 12 mois avant la date de jury + 6 mois, si oui on met diplôme X
    5. Si aucun poste trouvé on met "Non répondant".
    """

    row["6 mois - Date de l'enquête dans les 6 mois - datetime"] = pd.to_datetime(row["6 mois - Date de l'enquête dans les 6 mois"], errors="coerce", format="%d-%m-%Y")
    row["6 mois - Date de l'enquête dans les 6 mois - datetime"] = (row["6 mois - Date de l'enquête dans les 6 mois - datetime"].replace(day=1))
    row["6 mois - Situation"] = ""
    row["6 mois - Poste numero"] = ""
    row["6 mois - Diplôme numero"] = ""

    # Règle 1 : Non répondant = Pas de scraping réaliser sur ces personnes
    if (pd.isna(row["Poste 1"]) and pd.isna(row["École 1"]) and pd.isna(row["Diplôme 1"])) or pd.isna(row["Lien profil linkedin"]):
        return ["Non répondant", "", row["6 mois - Poste numero"], row["6 mois - Diplôme numero"]]


    # Règle 2 : on cherche le poste actif à 6 mois
    row["6 mois - Date poste"] = cherche_poste_actif_6_mois(row) # Liste des postes actifs à 6 mois avec le nom des colonnes date (ex. ['Date poste 2.1'])
        # Remplace "Date poste X" par "Poste X"
    if isinstance(row["6 mois - Date poste"], list):
        row["6 mois - Poste colonne"] = [s.replace("Date p", "P") for s in row["6 mois - Date poste"]]
    else:
        row["6 mois - Poste colonne"] = row["6 mois - Date poste"]
    row["6 mois - Poste"] = remplacer_colonne_poste_par_poste(row)


        # Cherche la colonne contenant le type de contrat associée au poste trouvé
    if isinstance(row["6 mois - Date poste"], list):
        row["6 mois - Poste numero"] = [s.replace("Date poste ", "") for s in row["6 mois - Date poste"]]
    else:
        row["6 mois - Poste numero"] = row["6 mois - Date poste"]


    #------------------------------------------------#
    poste = str(row["6 mois - Poste"]).lower()
    contrat_entreprise = str(trouver_contrat_entreprise_associe(row) or "").lower()
    #------------------------------------------------#


    #Règle 3 : cas alternance / stage
        # Si le contrat ou le poste contient 'alternance' ou 'stagiaire' alors cherche le diplôme en cours
    if any(word in contrat_entreprise for word in EDUCATION_KEYWORDS) or any(word in poste for word in EDUCATION_KEYWORDS):
        
        # Cherche le diplôme actif à 6 mois
        diplome = cherche_diplome_actif_6_mois(row)
        
        # Remplace "Date diplôme X" par "Diplôme X"
        if isinstance(diplome, list):
            row["6 mois - Diplôme numero"] = [s.replace("Date diplôme", "Diplôme") for s in diplome]
        else:
            row["6 mois - Diplôme numero"] = diplome


        # --- Sécurisation du champ "Poste numero" ---
        poste_nums = row.get("6 mois - Poste numero", None)
        if poste_nums is None:
            return ["Inactif / Recherche d'emploi", "", row["6 mois - Poste numero"], row["6 mois - Diplôme numero"]]

        if not isinstance(poste_nums, list):
            poste_nums = [poste_nums]

        match_found = False  # drapeau de contrôle

        for poste_num in poste_nums:
            if not isinstance(poste_num, str):
                continue

            date_debut = row.get(f"Date poste {poste_num}_Début")
            date_fin = row.get(f"Date poste {poste_num}_Fin")

            # Convertit les dates si besoin
            date_debut = pd.to_datetime(date_debut, errors="coerce")
            date_fin = pd.to_datetime(date_fin, errors="coerce")

            if pd.isna(date_debut) or pd.isna(date_fin) or date_fin < date_debut:
                continue

            duree = date_fin - date_debut

            # Vérifie alternance
            if (
                duree <= pd.Timedelta(days=365*3)
                and (any(word in contrat_entreprise for word in ALTERNANCE_KEYWORDS)
                    or any(word in poste for word in ALTERNANCE_KEYWORDS))
            ):
                row["6 mois - Poste"] = remplacer_colonne_diplome_par_diplome(row)
                
                if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "":
                    row["6 mois - Poste"] = "Diplôme non renseigné"
                row["6 mois - Situation"] = "ecole"
                match_found = True
                break

            # Vérifie stage
            elif (
                duree <= pd.Timedelta(days=30*6)
                and (any(word in contrat_entreprise for word in STAGE_KEYWORDS)
                    or any(word in poste for word in STAGE_KEYWORDS))
            ):
                row["6 mois - Poste"] = remplacer_colonne_diplome_par_diplome(row)
                if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "":
                    row["6 mois - Poste"] = "Diplôme non renseigné"
                row["6 mois - Situation"] = "ecole"
                match_found = True
                break

        if not match_found:
            row["6 mois - Poste"] = "Inactif / Recherche d'emploi"


    # Règle 4 : cas pause professionnelle
    if any(word in poste for word in PAUSE_KEYWORDS):
        row["6 mois - Poste"] = "Inactif"
        row["6 mois - Situation"] = "pause"
    


    # Règle 5 : Si poste est toujours vide on regarde si formation en cours de moins de 6 mois à la date de jury + 6 mois, si oui on met diplôme X
    if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "":

        # Cherche quel diplôme est a débuté maximum 6 mois avant date de jury + 6 mois
        diplome_actif = cherche_diplome_actif_6_mois(row)

                # Remplace "Date diplôme X" par "Diplôme X"
        if isinstance(diplome_actif, list):
            row["6 mois - Diplôme numero"] = [s.replace("Date diplôme", "Diplôme") for s in diplome_actif]
        else:
            row["6 mois - Diplôme numero"] = diplome_actif

    
        if isinstance(diplome_actif, list) and len(diplome_actif) > 0:
            diplome_actif = diplome_actif[0]

            if row[diplome_actif + "_Fin"] > row["6 mois - Date de l'enquête dans les 6 mois - datetime"] and (row[diplome_actif + "_Début"] > (row["6 mois - Date de l'enquête dans les 6 mois - datetime"] - pd.DateOffset(months=12))):
                row["6 mois - Poste"] = remplacer_colonne_diplome_par_diplome(row) # Remplace par le nom du diplôme
                # cas où il a bien une ecole de renseignée mais pas de diplôme 
                if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "":
                    row["6 mois - Poste"] = "Diplôme non renseigné"
                row["6 mois - Situation"] = "ecole"
            else:
                row["6 mois - Poste"] = "Inactif / Recherche d'emploi"


    # Règle 7 : cas indépendant
    if any(word in poste for word in INDEPENDANT_KEYWORDS) or any(word in contrat_entreprise for word in INDEPENDANT_KEYWORDS):
        row["6 mois - Situation"] = "independant"


    # Règle 8 : En recherche d'emploi
    if any(word in poste for word in RECHERCHE_EMPLOI) or any(word in contrat_entreprise for word in RECHERCHE_EMPLOI):
        row["6 mois - Situation"] = "recherche emploi"
        row["6 mois - Poste"] = "En recherche d'emploi"


    # Règle 6 : aucun poste trouvé
    if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "":
        row["6 mois - Poste"] = "Inactif / Recherche d'emploi"


    return [row["6 mois - Poste"], row["6 mois - Situation"], row["6 mois - Poste numero"], row["6 mois - Diplôme numero"]]

In [28]:
# Application des règles métier pour déterminer l'entreprise 6 mois

def trouver_entreprise_associe(row):
    """
    Trouve la colonne 'Nom entreprise + type contrat X' associée au poste trouvé.
    """

    poste_numero = row.get("6 mois - Poste numéro", None)

    # 1️⃣ Identifier la clé du poste
    if isinstance(poste_numero, (list, np.ndarray)) and len(poste_numero) > 0:
        poste_key = str(poste_numero[0])
    elif isinstance(poste_numero, str) and len(poste_numero.strip()) > 0:
        poste_key = poste_numero.strip()
    else:
        return None  # Rien à traiter

    # 2️⃣ Gérer les cas "X.X" → "Poste X" sinon "X" → "Nom entreprise + type contrat X"
    if '.' in poste_key:
        col_name = "Poste " + poste_key.split('.')[0]
    else:
        col_name =  "Nom entreprise + type contrat " + poste_key

    # 3️⃣ Vérifier que la colonne existe et n'est pas NaN
    if col_name not in row:
        return None
    else:
        return str(row[col_name])


def remplacer_colonne_ecole_par_ecole(row):
    """
    Remplace le nom de la colonne de l'école par le nom de l'école réelle.
    Si la colonne correspondante n'existe pas ou est vide, renvoie 'École non renseigné'.
    """

    diplome_numero = row.get("6 mois - Diplôme numéro", None)

    # 1 Gérer tous les cas possibles
    if diplome_numero is None or (isinstance(diplome_numero, float) and pd.isna(diplome_numero)):
        #return "coucou1"
        return "École non renseigné"
    if isinstance(diplome_numero, (list, tuple, np.ndarray)) and len(diplome_numero) > 0:
        ecole = str(diplome_numero[0]).replace("Diplôme", "École").strip()
    elif isinstance(diplome_numero, str) and diplome_numero.strip() != "":
        ecole = diplome_numero.replace("Diplôme", "École").strip()
    else:
        #return "coucou2"
        return "École non renseigné"

    # 2 Retourner la valeur si elle existe
    if ecole in row and pd.notna(row[ecole]) and str(row[ecole]).strip() != "":
        return str(row[ecole]).strip()
    else:
        #return "coucou3"
        return "École non renseigné"


def determine_6_mois_entreprise(row):
    """
    Détermination de l'entreprise à 6 mois selon les règles métier :

    1. Si "6 mois - Poste" est vide, entreprise est vide.
    2. Si "6 mois - Poste" est "Inactif" alors entreprise est vide.
    3. Sinon on cherche "Nom entreprise + type contrat" associée au poste à 6 mois et on le nettoie :
        - Si "Poste X.X" et non vide, on prend "Poste X"(Ici que se trouve le nom entreprise seul)
        - Sinon on prend "Nom entreprise + type contrat X"
    4. Si l'entreprise ou le contrat ou le poste actuel contient des mots clés (ex. 'freelance', 'indépendant', 'self-employed'), on met "Entrepreneur".
    5. Si "l'entreprise ou le contrat contient des mots clés (ex.'alternance' ou 'stagiaire'), on met "École X" (le diplôme en cours à 6 mois).
    6. Si poste est toujours vide on regarde si formation en cours qui a commencé moins de 6 mois avant la date de jury + 6 mois, si oui on met Ecole X
    """

    # Règle 1 : "6 mois - poste actuel" est vide ou non répondant
    if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "" or "non répondant" in row["6 mois - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "6 mois - poste actuel" est inactif
    if "inactif" in row["6 mois - Poste"].lower():
        return ""


    #------------------------------------------------#
    entreprise = trouver_entreprise_associe(row)
    entreprise = extract_entreprise(entreprise)
    #------------------------------------------------#


    #Règle 4 : cas freelance / indépendant, Si l'entreprise ou le contrat contient des mots clés liés aux indépendants alors "Entrepreneur"
    if pd.notna(row["6 mois - Situation"]) and "independant" in row["6 mois - Situation"].lower():
        entreprise = "Entrepreneur"
    

    # Règle 5 : cas alternance / stage, # Si le contrat ou le poste contient 'alternance' ou 'stagiaire' alors cherche l'école en cours
    if pd.notna(row["6 mois - Situation"]) and "ecole" in row["6 mois - Situation"].lower():
        entreprise = remplacer_colonne_ecole_par_ecole(row)


    # Règle 6 : cas recherche emploi
    if pd.notna(row["6 mois - Situation"]) and "recherche emploi" in row["6 mois - Situation"].lower():
        return ""
        
    
    return entreprise

In [29]:
# Application des règles métier pour déterminer le contrat à 6 mois

def trouver_contrat_associe(row):
    """
    Trouver les deux colonnes "Nom entreprise + type contrat X" et "Nom entreprise + type contrat X.X" associée au poste trouvé où se trouve possiblement l'entreprise
    """

    poste_numero = row.get("6 mois - Poste numéro", None)

    # 1️⃣ Identifier la clé du poste
    if isinstance(poste_numero, (list, np.ndarray)) and len(poste_numero) > 0:
        poste_key = str(poste_numero[0])
    elif isinstance(poste_numero, str) and len(poste_numero.strip()) > 0:
        poste_key = poste_numero.strip()
    else:
        return None  # Rien à traiter

    # Poste_key_2 par défaut = poste_key (sécurisation)
    poste_key_2 = poste_key
    # Si poste_key contient un point, on prend seulement la partie avant
    if '.' in poste_key:
        poste_key_2 = poste_key.split('.')[0]

    # Retourne la concaténation sécurisée
    return (
        str(row.get("Poste " + poste_key, "")).lower() + " " +
        str(row.get("Nom entreprise + type contrat " + poste_key, "")).lower() + " " +
        str(row.get("Nom entreprise + type contrat " + poste_key_2, "")).lower()
    )


def determine_6_mois_contrat(row):
    """
    Détermination du contrat à 6 mois selon les règles métier :

    1. Si "6 mois - Poste" est vide, contrat est "Non répondant".
    2. Si "6 mois - Poste" est "Inactif" alors contrat est "Non répondant".
    3. Vérifie la présence de mots clés dans le champ contrat pour déterminer le type de contrat
    4. Si l'entreprise ou le contrat ou le poste actuel contient des mots clés (ex. 'freelance', 'indépendant', 'self-employed', 'fondateur'...), => "CDI".
    5. Si l'entreprise ou le contrat ou le poste actuel contient des mots clés (ex. 'alternance', 'stagiaire'...), => ""
    """

    # Règle 1 : "6 mois - poste actuel" est vide ou non répondant
    if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "" or "non répondant" in row["6 mois - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "6 mois - poste actuel" est inactif
    if "inactif" in row["6 mois - Poste"].lower():
        return ""


    # Règle 3 : on cherche "Nom entreprise + type contrat" associée au poste à 6 mois
    contrat_entreprise = trouver_contrat_associe(row)
    contrat = extract_contrat(contrat_entreprise)


    #Règle 4 : cas freelance / indépendant, Si l'entreprise ou le contrat contient des mots clés liés aux indépendants alors "CDI"
    if pd.notna(row["6 mois - Situation"]) and "independant" in row["6 mois - Situation"].lower():
        return "CDI"

    
    # Règle 5 : cas alternance / stage, Si l'entreprise ou le contrat ou le poste contient 'alternance' ou 'stagiaire' alors ""
    if pd.notna(row["6 mois - Situation"]) and "ecole" in row["6 mois - Situation"].lower():
        return ""


    numero_poste = extract_single_value(row["6 mois - Poste numéro"])


    # Règle 6 : Si le contrat est un CDD de plus de 18 mois on le transforme en CDI
    if contrat == "CDD":
        if numero_poste is not None:
            date_debut = row[f"Date poste {numero_poste}_Début"] if pd.notna(row[f"Date poste {numero_poste}_Début"]) else None
            date_fin = row[f"Date poste {numero_poste}_Fin"] if pd.notna(row[f"Date poste {numero_poste}_Fin"]) else None
            if date_debut and date_fin and (date_fin - date_debut) > pd.Timedelta(days=18*30):        
                contrat = "CDI"
            else:
                contrat = "CDD"


    # Règle 6 : Si le contrat est vide et que le poste a commencé il y plus de 4 mois on le transforme en CDI sinon CDD
    if contrat == "":
        if numero_poste is not None:
            date_debut = row[f"Date poste {numero_poste}_Début"] if pd.notna(row[f"Date poste {numero_poste}_Début"]) else None
            date_fin = row[f"Date poste {numero_poste}_Fin"] if pd.notna(row[f"Date poste {numero_poste}_Fin"]) else None
            if date_debut and date_fin and (date_fin - date_debut) > pd.Timedelta(days=4*30):
                contrat = "CDI"
            else:
                contrat = "CDD"


    # Règle 7 : cas recherche emploi
    if pd.notna(row["6 mois - Situation"]) and "recherche emploi" in row["6 mois - Situation"].lower():
        return ""
    

    # Règle 8 : Si plusieurs poste dans la même entreprise alors CDI
    if numero_poste is not None and "." in numero_poste:
        contrat = "CDI"


    return contrat

In [30]:
# Application des règles métier pour déterminer le statut cadre / non-cadre à 6 mois

def determine_6_mois_statut(row):
    """
    Détermination du statut cadre / non-cadre à 6 mois selon les règles métier :

    1. Si "6 mois - Poste" est vide, statut est "Non répondant".
    2. Si "6 mois - Poste" est "Inactif" alors statut est "".
    3. Si c'est une alternance ou stage, statut est "".
    4. Vérifie la présence de mots clés dans le champ "6 mois - Poste" et "6 mois - Type de contrat"  pour déterminer le type de statut
    """

    # Règle 1 : "6 mois - poste actuel" est vide ou non répondant
    if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "" or "non répondant" in row["6 mois - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "6 mois - poste actuel" est inactif
    if "inactif" in row["6 mois - Poste"].lower():
        return ""
    

    # Règle 4 : Vérifie la présence de mots clés dans le champ "6 mois - Poste" et "Nom entreprise + type contrat X" si statut non trouvé
    statut = extract_statut_cadre(str(row["6 mois - Poste"]))


    # Règle 5 :  cas freelance / indépendant
    if pd.notna(row["6 mois - Situation"]) and "independant" in row["6 mois - Situation"].lower():
        statut = "Cadre"


    # Règle 3 : Si c'est une alternance ou stage, statut est "".
    if "ecole" in row["6 mois - Situation"].lower():
        statut = ""


    # Règle 4 : cas recherche emploi
    if pd.notna(row["6 mois - Situation"]) and "recherche emploi" in row["6 mois - Situation"].lower():
        return ""

    return statut

In [31]:
# Application des règles métier pour déterminer la quotité de travail à 6 mois

def determine_6_mois_quotite(row):
    """
    Détermination de la quotité de travail actuel selon les règles métier :

    1. Si "6 mois - Poste" est vide, quotité est "".
    2. Si "6 mois - Poste" est "Inactif" alors quotité est "".
    3. Si "Nom entreprise + type contrat" est auto entrepreneur / freelance alors quotité est 100
    4. Vérifie la présence de mots clés dans le champ "Nom entreprise + type contrat 1" et "Nom entreprise + type contrat 1.1" pour déterminer la quotité
    """

    # Règle 1 : "6 mois - poste actuel" est vide ou non répondant
    if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "" or "non répondant" in row["6 mois - Poste"].lower():
        return "Non répondant"


    # Règle 2 : "6 mois - poste actuel" est inactif
    if "inactif" in row["6 mois - Poste"].lower():
        return ""


    # Règle 3 :  cas freelance / indépendant
    if pd.notna(row["6 mois - Situation"]) and "independant" in row["6 mois - Situation"].lower():
        quotite = "100"


    # Règle 4 : Vérifie la présence de mots clés
    quotite = trouver_contrat_associe(row)
    quotite = extract_quotite(quotite)


    # Règle 6 : Si il y a qu'un seul poste actif alors on met 100
    if quotite == "":
        poste_actif_6_mois = row["6 mois - Poste numéro"]
        nb_elements = len(poste_actif_6_mois) if isinstance(poste_actif_6_mois, list) else 1
        if nb_elements == 1:
            quotite = "100"
        elif nb_elements > 1:
            quotite = "Plusieurs postes en même temps"
        else:
            quotite = None
        

    # Règle 5 : Si c'est une alternance ou stage, statut est "".
    if pd.notna(row["6 mois - Situation"]) and "ecole" in row["6 mois - Situation"].lower():
        quotite = ""


    # Règle 6 : cas recherche emploi
    if pd.notna(row["6 mois - Situation"]) and "recherche emploi" in row["6 mois - Situation"].lower():
        return ""


    return quotite

In [32]:
# Application des règles métier pour déterminer la situation à 6 mois après la certification

def determine_6_mois_situation_apres_certification(row):
    """
    Détermination de la situation après la certification selon les règles métier :

    1. Si "6 mois - Poste" est vide, situation est "Non répondant".
    2. Si "6 mois - Poste" est "Inactif" alors situation est "Inactif".
    -----------------
    3. Si le poste ou contrat ou entreprise contient des mots clés liés à une pause professionnelle (ex: "pause pro", "travel") on met "Inactif".
    4. Si le poste ou contrat ou entreprise contient des mots clés liés à l'alternance ou stage (ex: "alternance", "stagiaire", "apprenticeship", "etudiant", "étudiant", "alternant") on met "En formation y compris alternance".
    
    
    
    5. Gérer les cas recherche d'emploi
    6. Gérer les cas autres
    
    
    
    7. Sinon on met "Actif occupé hors alternance".
    """

    # Règle 1 : "6 mois - poste actuel" est vide
    if pd.isna(row["6 mois - Poste"]) or row["6 mois - Poste"] == "" or "non répondant" in row["6 mois - Poste"].lower():
        return "Non répondant"


    # Règle 2 : Si "6 mois - Poste" est "Inactif / Recherche d'emploi" alors situation est "Inactif / Recherche d'emploi".
    if row["6 mois - Poste"] == "Inactif / Recherche d'emploi":
        return "Inactif / Recherche d'emploi"


    # Règle 3 : Si "6 mois - Poste" est "Inactif" alors situation est "Inactif".
    if row["6 mois - Poste"] == "Inactif":
        return "Inactif"
    

    # Règle 4 : Si considéré comme pause alors ""
    if pd.notna(row["6 mois - Situation"]) and "pause" in row["6 mois - Situation"].lower():
        return "Inactif"


    # Règle 5 : cas alternance / stage
    if pd.notna(row["6 mois - Situation"]) and "ecole" in row["6 mois - Situation"].lower():
        return "En formation y compris alternance"


    # Règle 6 : cas recherche emploi
    if pd.notna(row["6 mois - Situation"]) and "recherche emploi" in row["6 mois - Situation"].lower():
        return "En recherche d'emploi"
    

    # Règle 7 : sinon
    return "Actif occupé hors alternance"

### Situation avant la certification

In [33]:
# Application des règles métier pour déterminer "intitulé exact de la dernière certification obtenue"

def determine_intitule_derniere_certification_obtenue(row):
    """
    Détermination de l'intitulé exact de la dernière certification obtenue selon les règles métier :

    1. Détermine la dernière certification obtenue avant la date de jury.
    """

    date_jury = row["Date de la décision d'attribution de la certification"]
    annee_date_jury = date_jury.year
    

    # Règle 1 : Détermine la dernière certification obtenue avant la date de jury.


    # Récupère les dates de fin de diplôme
    diplomes = {
        "1": row.get("Date diplôme 1_Fin"),
        "2": row.get("Date diplôme 2_Fin"),
        "3": row.get("Date diplôme 3_Fin")
    }

    # Filtre : ne garder que les diplômes dont l'année < année du jury
    diplomes_valides = {
        k: v for k, v in diplomes.items()
        if pd.notna(v) and v.year < annee_date_jury
    }

    if len(diplomes_valides) == 0:
        return [None, None]  # aucun diplôme avant ou pendant l'année du jury

    # Sélectionne la date la plus récente
    dernier_key = max(diplomes_valides, key=lambda k: diplomes_valides[k])

    # Récupère le nom du diplôme associé
    dernier_diplome_nom = row.get(f"Diplôme {dernier_key}")

    # Cas particulier : si le champ existe mais contient une chaîne vide, None ou NaN
    if (
        dernier_diplome_nom is None
        or (isinstance(dernier_diplome_nom, float) and pd.isna(dernier_diplome_nom))
        or str(dernier_diplome_nom).strip() == ""
    ):
        dernier_diplome_nom = "Diplôme non renseigné"



    return [dernier_diplome_nom, dernier_key]

In [34]:
# Application des règles métier pour déterminer "Niveau de qualification de la dernière certification obtenue (sans niveau, de 2 à 8, autre, non répondant)"

def determine_niveau_qualification_derniere_certification_obtenue(row):
    """
    Détermination du niveau de qualification de la dernière certification obtenue (sans niveau, de 2 à 8, autre, non répondant) selon les règles métier :
    1. Détermine le niveau de qualification de la dernière certification obtenue avant la date de jury par rapport aux mots clés
    """

    certification = str(row.get("Avant - Intitulé formation")).lower()


    # Règle 1 : Détermine le niveau de qualification de la dernière certification obtenue avant la date de jury par rapport aux mots clés
    niveau_qualification = extract_niveau_qualification(certification)


    return niveau_qualification

In [35]:
# Application des règles métier pour déterminer "le dernier métier exercé"

def trouver_entreprise_contrat_dernier_metier(poste_numero, row):
    """
    Trouver les deux colonnes "Nom entreprise + type contrat X" et "Nom entreprise + type contrat X.X" associée au poste trouvé où se trouve possiblement l'entreprise
    """

    # 1️⃣ Identifier la clé du poste
    if isinstance(poste_numero, (list, np.ndarray)) and len(poste_numero) > 0:
        poste_key = str(poste_numero[0])
    elif isinstance(poste_numero, str) and len(poste_numero.strip()) > 0:
        poste_key = poste_numero.strip()
    else:
        return None  # Rien à traiter

    # Poste_key_2 par défaut = poste_key (sécurisation)
    poste_key_2 = poste_key
    # Si poste_key contient un point, on prend seulement la partie avant
    if '.' in poste_key:
        poste_key_2 = poste_key.split('.')[0]

    # Retourne la concaténation sécurisée
    return (
        str(row.get("Poste " + poste_key, "")).lower() + " " +
        str(row.get("Nom entreprise + type contrat " + poste_key, "")).lower() + " " +
        str(row.get("Nom entreprise + type contrat " + poste_key_2, "")).lower()
    )
    

def determine_dernier_metier_excerce(row):
    """
    Détermination le dernier métier exercé avant la certification selon les règles métier :
    1. Métier doit commencer avant début de la formation ET durant minimum 1 ans
    """

    date_jury = pd.to_datetime(row.get("Date de la décision d'attribution de la certification"), errors="coerce")
    if pd.isnull(date_jury):
        return [None, None]

    certification_debut_theorique = date_jury - pd.DateOffset(years=2)
    limit_debut_metier = date_jury - pd.DateOffset(years=3)

    colonnes_debut = [c for c in row.index if ("poste" in c and "Début" in c)]
    colonnes_fin = [c for c in row.index if ("poste" in c and "Fin" in c)]


    for col_debut in colonnes_debut:
        # Identifier le numéro du poste (ex: "1", "2.1")
        numero = (
            col_debut.replace("Date poste", "")
            .replace("_Début", "")
            .replace("Début", "")
            .strip()
        )

        # Trouver la colonne de fin correspondante
        col_fin_candidates = [c for c in colonnes_fin if numero in c]
        if not col_fin_candidates:
            continue
        col_fin = col_fin_candidates[0]

        # Conversion en datetime
        debut = pd.to_datetime(row[col_debut], errors="coerce")
        fin = pd.to_datetime(row[col_fin], errors="coerce")

        if pd.isnull(debut) or pd.isnull(fin):
            continue
        
        duree = fin - debut

        # Application des règles
        if (debut < limit_debut_metier) and (fin >= certification_debut_theorique) and (duree >= pd.Timedelta(days=365)):
            entreprise_contrat = trouver_entreprise_contrat_dernier_metier(numero, row)
            poste = row.get(f"Poste {numero}")
            
            if (entreprise_contrat and any(word in entreprise_contrat for word in EDUCATION_KEYWORDS)) or (poste and any(word in poste for word in EDUCATION_KEYWORDS)):
                return [None, None]
            else:
                return [poste, numero]


    return [None, None]

In [36]:
# Application des règles métier pour déterminer "la durée du dernier métier exercé"

def determine_duree_dernier_metier_excerce(row):
    """
    Détermination de la durée du dernier métier exercé avant la certification
    """

    if pd.isna(row["Avant - Métier"]) or row["Avant - Métier"] is None:
        return ""


    poste_numero = row.get("Avant - Métier poste numero")
    if pd.isna(poste_numero) or poste_numero is None:
        return None

    debut_poste = f"Date poste {poste_numero}_Début"
    fin_poste = f"Date poste {poste_numero}_Fin"
    date_debut_certification_theorique = row["Date de la décision d'attribution de la certification"] - pd.DateOffset(years=2)


    debut_poste = pd.to_datetime(row.get(debut_poste), errors="coerce")
    fin_poste = pd.to_datetime(row.get(fin_poste), errors="coerce")
    
    
    if pd.isna(debut_poste) or pd.isna(fin_poste):
        return None


    if fin_poste > date_debut_certification_theorique:
        duree_poste = date_debut_certification_theorique - debut_poste
    else:
        duree_poste = fin_poste - debut_poste
    
    duree_annees = duree_poste / pd.Timedelta(days=365.25)  # conversion en années

    return round(duree_annees)

In [37]:
# Application des règles métier pour déterminer "l'entreprise du dernier métier exercé" 
def trouver_entreprise_dernier_metier(row):
    """
    Trouve la colonne 'Nom entreprise + type contrat X' associée au poste trouvé.
    """

    poste_numero = row.get("Avant - Métier poste numero", None)

    # 1️⃣ Identifier la clé du poste
    if isinstance(poste_numero, (list, np.ndarray)) and len(poste_numero) > 0:
        poste_key = str(poste_numero[0])
    elif isinstance(poste_numero, str) and len(poste_numero.strip()) > 0:
        poste_key = poste_numero.strip()
    else:
        return None  # Rien à traiter

    # 2️⃣ Gérer les cas "X.X" → "Poste X" sinon "X" → "Nom entreprise + type contrat X"
    if '.' in poste_key:
        col_name = "Poste " + poste_key.split('.')[0]
    else:
        col_name =  "Nom entreprise + type contrat " + poste_key

    # 3️⃣ Vérifier que la colonne existe et n'est pas NaN
    if col_name not in row:
        return None
    else:
        return str(row[col_name])


def determine_entreprise_dernier_metier_excerce(row):
    """
    Détermination de l'entreprise du dernier métier exercé avant la certification
    """

    if pd.isna(row["Avant - Métier"]) or row["Avant - Métier"] is None:
        return ""

    entreprise_contrat = trouver_entreprise_dernier_metier(row)

    entreprise = extract_entreprise(entreprise_contrat)


    if pd.isna(entreprise) or entreprise is None:
        return ""
    
    
    return entreprise

In [38]:
def determine_situation_avant_parcours_certifiant(row):
    """
    Détermination de l'intitulé exact de la dernière certification obtenue selon les règles métier :

    1.Si aucune formation ni aucun métier n’est détecté → 
    🔹 Sortie : « Demander à l’école » 

    2.Si la dernière formation s’est terminée moins d’un an avant le début de la certification → 
    🔹 Sortie : « En formation y compris alternance » 

    3.Si un poste s’est terminé moins d’un an avant le début de la certification → 
    🔹 Sortie : « Actif occupé hors alternance » 
    """

    # --- Règle 1 : Si formation et métier sont vides
    if (
        pd.isna(row.get("Avant - Intitulé formation")) or str(row.get("Avant - Intitulé formation")).strip() == ""
    ) and (
        pd.isna(row.get("Avant - Métier")) or str(row.get("Avant - Métier")).strip() == ""
    ):
        return "Demander à l'école"

    # --- Récupération sécurisée des champs utilisés ensuite ---
    formation_num = row.get("Avant - Formation numero")
    metier_num = row.get("Avant - Métier poste numero")

    # Dates récupérées de manière sécurisée
    date_fin_derniere_formation = row.get(f"Date diplôme {formation_num}_Fin") if formation_num else None
    date_jury = row.get("Date de la décision d'attribution de la certification")
    date_fin_dernier_metier = row.get(f"Date poste {metier_num}_Fin") if metier_num else None

    # --- Convertir en années si possible ---
    def get_year_safe(x):
        if pd.notnull(x):
            try:
                return pd.to_datetime(x).year
            except Exception:
                return None
        return None

    annee_fin_formation = get_year_safe(date_fin_derniere_formation)
    annee_jury = get_year_safe(date_jury)
    annee_fin_metier = get_year_safe(date_fin_dernier_metier)

    # Si on ne peut rien exploiter
    if annee_jury is None:
        return "Demander à l'école"

    # Date de début théorique formation = jury - 3 ans
    annee_debut_theorique_formation = annee_jury - 3

    # --- Règle 2 : Formation terminée < 1 an avant début certif ---
    if annee_fin_formation is not None:
        ecart_formation = annee_debut_theorique_formation - annee_fin_formation
        if ecart_formation <= 1:
            return "En formation y compris alternance"

    # --- Règle 3 : Si formation vide → regarder le métier ---
    if annee_fin_metier is not None:
        ecart_metier = annee_debut_theorique_formation - annee_fin_metier
        if ecart_metier <= 1:
            return "Actif occupé hors alternance"

    # Si aucune règle ne s’applique
    return "Demander à l'école"

### Création des colonnes avec les fonctions

In [39]:
#----------------------- LONG TERME -----------------------#
# Détermine le poste actuel selon les règles métier
EPSI_scraping_result["Long terme - Poste"] = EPSI_scraping_result.apply(determine_long_terme_poste, axis=1).str[0]
EPSI_scraping_result["Long terme - Situation"] = EPSI_scraping_result.apply(determine_long_terme_poste, axis=1).str[1]
EPSI_scraping_result["Long terme - Poste numéro"] = EPSI_scraping_result.apply(determine_long_terme_poste, axis=1).str[2]

# Détermine l'entreprise actuelle selon les règles métier
EPSI_scraping_result["Long terme - Entreprise"] = EPSI_scraping_result.apply(determine_long_terme_entreprise, axis=1)

# Détermine le type de contrat actuel selon les règles métier
EPSI_scraping_result["Long terme - Type de contrat"] = EPSI_scraping_result.apply(determine_long_terme_contrat, axis=1)

# Détermine le statut cadre / non-cadre actuel selon les règles métier
EPSI_scraping_result["Long terme - Statut cadre / non-cadre"] = EPSI_scraping_result.apply(determine_long_terme_statut, axis=1)

# Détermine la quotité de travail actuel selon les règles métier
EPSI_scraping_result["Long terme - Quotité de travail"] = EPSI_scraping_result.apply(determine_long_terme_quotite, axis=1)

# Détermine la situation après la certification selon les règles métier
EPSI_scraping_result["Long terme - Situation après la certification"] = EPSI_scraping_result.apply(determine_long_terme_situation_apres_certification, axis=1)

# Détermine le nombre d'années après l'obtention de la certification selon les règles métier
EPSI_scraping_result["Long terme - Nombre d'années après l'obtention de la certification"] = EPSI_scraping_result.apply(determine_long_terme_annees_apres_certification, axis=1)


#----------------------- 6 MOIS -----------------------#
# Détermine la date de l'enquête dans les 6 mois selon les règles métier
EPSI_scraping_result["6 mois - Date de l'enquête dans les 6 mois"] = ((EPSI_scraping_result["Date de la décision d'attribution de la certification"] + pd.DateOffset(months=6))).dt.strftime("%d-%m-%Y")

# Détermine le à 6 mois selon les règles métier
EPSI_scraping_result["6 mois - Poste"] = EPSI_scraping_result.apply(determine_6_mois_poste, axis=1).str[0]
EPSI_scraping_result["6 mois - Situation"] = EPSI_scraping_result.apply(determine_6_mois_poste, axis=1).str[1]
EPSI_scraping_result["6 mois - Poste numéro"] = EPSI_scraping_result.apply(determine_6_mois_poste, axis=1).str[2]
EPSI_scraping_result["6 mois - Diplôme numéro"] = EPSI_scraping_result.apply(determine_6_mois_poste, axis=1).str[3]

# Détermine l'entreprise à 6 mois selon les règles métier
EPSI_scraping_result["6 mois - Entreprise"] = EPSI_scraping_result.apply(determine_6_mois_entreprise, axis=1)

# Détermine le type de contrat à 6 mois selon les règles métier
EPSI_scraping_result["6 mois - Type de contrat"] = EPSI_scraping_result.apply(determine_6_mois_contrat, axis=1)

# Détermine le statut cadre / non-cadre à 6 mois selon les règles métier
EPSI_scraping_result["6 mois - Statut cadre / non-cadre"] = EPSI_scraping_result.apply(determine_6_mois_statut, axis=1)

# Détermine la quotité de travail à 6 mois selon les règles métier
EPSI_scraping_result["6 mois - Quotité de travail"] = EPSI_scraping_result.apply(determine_6_mois_quotite, axis=1)

# Détermine la situation à 6 mois selon les règles métier
EPSI_scraping_result["6 mois - Situation après la certification"] = EPSI_scraping_result.apply(determine_6_mois_situation_apres_certification, axis=1)


#----------------------- AVANT CERTIFICATION -----------------------#
# Détermine le dernier diplôme / certification obtenu avant la certification selon les règles métier
EPSI_scraping_result["Avant - Intitulé formation"] = EPSI_scraping_result.apply(determine_intitule_derniere_certification_obtenue, axis=1).str[0]
EPSI_scraping_result["Avant - Formation numero"] = EPSI_scraping_result.apply(determine_intitule_derniere_certification_obtenue, axis=1).str[1]

# Détermine le niveau de qualification obtenu avant la certification selon les règles métier
EPSI_scraping_result["Avant - Niveau qualification"] = EPSI_scraping_result.apply(determine_niveau_qualification_derniere_certification_obtenue, axis=1)

# Détermine le métier exercé avant la certification selon les règles métier
EPSI_scraping_result["Avant - Métier"] = EPSI_scraping_result.apply(determine_dernier_metier_excerce, axis=1).str[0]
EPSI_scraping_result["Avant - Métier poste numero"] = EPSI_scraping_result.apply(determine_dernier_metier_excerce, axis=1).str[1]

# Détermine la durée du métier exercé avant la certification selon les règles métier
EPSI_scraping_result["Avant - Métier duree"] = EPSI_scraping_result.apply(determine_duree_dernier_metier_excerce, axis=1)

# Détermine l'entreprise du métier exercé avant la certification selon les règles métier
EPSI_scraping_result["Avant - Métier entreprise"] = EPSI_scraping_result.apply(determine_entreprise_dernier_metier_excerce, axis=1)


# Détermine la situation avant la certification selon les règles métier
EPSI_scraping_result["Avant - Situation avant la certification"] = EPSI_scraping_result.apply(determine_situation_avant_parcours_certifiant, axis=1)

# Création du fichier france compétence

## Sélection des colonnes provenant du traitement

In [40]:
france_competence = EPSI_scraping_result[[
                    "Avant - Intitulé formation",
                    "Avant - Niveau qualification",
                    "Avant - Métier",
                    "Avant - Métier duree",
                    "Avant - Métier entreprise",
                    "Avant - Situation avant la certification",
                    #--------------------------
                    "6 mois - Situation après la certification",
                    "6 mois - Poste",
                    "6 mois - Type de contrat",
                    "6 mois - Quotité de travail",
                    "6 mois - Entreprise",
                    "6 mois - Statut cadre / non-cadre",
                    "6 mois - Date de l'enquête dans les 6 mois",
                    #--------------------------
                    "Long terme - Situation après la certification",
                    "Long terme - Poste",
                    "Long terme - Type de contrat",
                    "Long terme - Quotité de travail",
                    "Long terme - Entreprise",
                    "Long terme - Statut cadre / non-cadre",
                    "Long terme - Nombre d'années après l'obtention de la certification",
                    "Lien profil linkedin",
                  ]
                  ]

In [41]:
# Détermine la date de l'enquête actualisée soit la date du scraping
france_competence["Long terme - Date de l'enquête actualisée"] = pd.to_datetime(DATE_LONG_TERME_COMPARAISON).date()
france_competence["Date de la décision d'attribution de la certification"] = EPSI_scraping_result["Date de la décision d'attribution de la certification"].dt.date
france_competence["Année d'obtention de la certification"] = EPSI_scraping_result["Année d'obtention de la certification"]
france_competence["Le cas échéant option choisie"] = EPSI_scraping_result["Le cas échéant option choisie"]
france_competence["Nom et Prénom du titulaire"] = EPSI_scraping_result["Nom et Prénom du titulaire"]
france_competence["Dénomination de l'organisme ayant assuré la formation (sauf candidat libre et reconnaissance des acquis)"] = EPSI_scraping_result["Dénomination de l'organisme ayant assuré la formation (sauf candidat libre et reconnaissance des acquis)"]
france_competence["Modalités pédagogiques (présentiel, distanciel, hybride, AFEST)"] = EPSI_scraping_result["Modalités pédagogiques (présentiel, distanciel, hybride, AFEST)"]
france_competence["Intitulé de la formation ayant préparé au projet ou à la certification (sauf candidat libre, reconnaissance des acquis et VAE)"] = EPSI_scraping_result["Intitulé de la formation ayant préparé au projet ou à la certification (sauf candidat libre, reconnaissance des acquis et VAE)"]
france_competence["Voie d'accès (formation initiale hors altenance, formation continue hors alternance, contrat d'apprentissage, contrat de professionnalisation, reconnaissance des acquis, VAE, candidat individuel, autre, non répondant)"] = EPSI_scraping_result["Voie d'accès (formation initiale hors altenance, formation continue hors alternance, contrat d'apprentissage, contrat de professionnalisation, reconnaissance des acquis, VAE, candidat individuel, autre, non répondant)"]

C:\Users\aurel\AppData\Local\Temp\ipykernel_9784\1370280609.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_competence["Long terme - Date de l'enquête actualisée"] = pd.to_datetime(DATE_LONG_TERME_COMPARAISON).date()
C:\Users\aurel\AppData\Local\Temp\ipykernel_9784\1370280609.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_competence["Date de la décision d'attribution de la certification"] = EPSI_scraping_result["Date de la décision d'attribution de la certification"].dt.date
C:\Users\

In [ ]:
def clean_str(value):
    """Nettoie une valeur texte (gère NaN, None, majuscules, espaces)."""
    if pd.isna(value):
        return ""
    return str(value).strip().lower()

def generer_a_verifier(row):
    conditions = []

    #---------------------- Avant -----------------------#


    #---------------------- Fonctions utilitaires -----------------------#
    def verifier_bloc(prefix):
        situation = clean_str(row.get(f"{prefix} - Situation"))
        situation_apres_parcours = clean_str(row.get(f"{prefix} - Situation après la certification (actif occupé hors alternance, en recherche d'emploi, en formation y compris alternance, inactif, autre, non répondant)"))
        poste = clean_str(row.get(f"{prefix} - Intitulé du poste occupé ou de l'activité indépendante (1)"))
        entreprise = clean_str(row.get(f"{prefix} - Nom de l'entreprise (1)"))
        contrat = clean_str(row.get(f"{prefix} - Type  de contrat ou de statut (CDI, CDD, CDDU, intérim, indépendant, autre, non répondant) (1)"))
        quotite = clean_str(row.get(f"{prefix} - Quotité de temps de travail lié au poste occupé ou à l'activité indépendante (en %)"))

        if situation_apres_parcours in ("inactif / recherche d'emploi", "inactif"):
            conditions.append(f"{prefix} - Inactif ou Recherche d'emploi")
        else:
            if poste == "diplôme non renseigné":
                conditions.append(f"{prefix} - Diplôme non renseigné")
            if entreprise == "école non renseigné":
                conditions.append(f"{prefix} - École non renseigné")
            if entreprise == "entrepreneur":
                conditions.append(f"{prefix} - Entrepreneur")
            if contrat == "" and situation not in ("ecole", "recherche emploi"):
                conditions.append(f"{prefix} - Manque contrat")
            if quotite == "plusieurs postes en même temps":
                conditions.append(f"{prefix} - Plusieurs postes en même temps")

    #---------------------- 6 mois -----------------------#
    verifier_bloc("6 mois")

    #---------------------- Long terme -----------------------#
    verifier_bloc("Long terme")

    return " / ".join(conditions) if conditions else None


# Application
france_competence_a_verifier = france_competence.copy()
france_competence_a_verifier["A vérifier"] = france_competence_a_verifier.apply(generer_a_verifier, axis=1)


In [69]:
france_competence_a_verifier[france_competence_a_verifier["A vérifier"].isin(["6 mois - Manque contrat / Long terme - Manque contrat", "6 mois - Manque contrat"])]

,Avant - Intitulé formation,Avant - Niveau qualification,Avant - Métier,Avant - Métier duree,Avant - Métier entreprise,Avant - Situation avant la certification,6 mois - Situation après la certification,6 mois - Poste,6 mois - Type de contrat,6 mois - Quotité de travail,...,Long terme - Date de l'enquête actualisée,Date de la décision d'attribution de la certification,Année d'obtention de la certification,Le cas échéant option choisie,Nom et Prénom du titulaire,Dénomination de l'organisme ayant assuré la formation (sauf candidat libre et reconnaissance des acquis),"Modalités pédagogiques (présentiel, distanciel, hybride, AFEST)","Intitulé de la formation ayant préparé au projet ou à la certification (sauf candidat libre, reconnaissance des acquis et VAE)","Voie d'accès (formation initiale hors altenance, formation continue hors alternance, contrat d'apprentissage, contrat de professionnalisation, reconnaissance des acquis, VAE, candidat individuel, autre, non répondant)",A vérifier
0,None,,None,,,Demander à l'école,Actif occupé hors alternance,Ingénieur développement,CDI,100,...,2025-09-30,2021-07-10,2021,NaN,AUGAGNEUR Alessandro,EPSI LYON,NaN,NaN,Contrat de professionnalisation,6 mois - Manque contrat / Long terme - Manque ...
1,Titre RNCP Niveau 6 – « Concepteur Développeur...,Niveau 6,Concepteur développeur informatique,2,DRA Technologies,En formation y compris alternance,Actif occupé hors alternance,Développeur applications mobiles,CDI,100,...,2025-09-30,2021-07-10,2021,NaN,BARRY Maxime,EPSI LYON,NaN,NaN,Contrat d'apprentissage,6 mois - Manque contrat / Long terme - Manque ...
2,"bachelor informatique, Informatique",Niveau 6,None,,,En formation y compris alternance,Actif occupé hors alternance,Ingénieur Développement et Support,CDI,100,...,2025-09-30,2021-07-10,2021,NaN,BASTION Gatien,EPSI LYON,NaN,NaN,Contrat de professionnalisation,6 mois - Manque contrat / Long terme - Manque ...
3,Bac + 3 Titre RNCP niveau 6 « Concepteur Dével...,Niveau 6,None,,,En formation y compris alternance,Actif occupé hors alternance,Consultant en intelligence décisionnelle,CDI,100,...,2025-09-30,2021-07-10,2021,NaN,BAUDOT Pierrick,EPSI LYON,NaN,NaN,Contrat de professionnalisation,6 mois - Manque contrat / Long terme - Manque ...
4,None,,None,,,Demander à l'école,Actif occupé hors alternance,Développeur Python,CDI,100,...,2025-09-30,2021-07-10,2021,NaN,BIRON Romain,EPSI LYON,NaN,NaN,Formation initiale hors alternance,6 mois - Manque contrat / Long terme - Manque ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204,"Master en informatique, Ingénierie informatique",Niveau 7,None,,,En formation y compris alternance,Actif occupé hors alternance,Développeur applications,CDI,100,...,2025-09-30,2024-11-07,2024,NaN,BALLANGER Alan,EPSI NANTES,NaN,NaN,NaN,6 mois - Manque contrat / Long terme - Manque ...
2205,Expert en informatique et systèmes d’informati...,,None,,,En formation y compris alternance,Actif occupé hors alternance,Ingénieur Full Stack 360,CDI,Plusieurs postes en même temps,...,2025-09-30,2024-11-07,2024,NaN,LE RAY Arthur,EPSI NANTES,NaN,NaN,NaN,6 mois - Manque contrat / Long terme - Manque ...
2206,None,,None,,,Demander à l'école,Non répondant,Non répondant,Non répondant,Non répondant,...,2025-09-30,2024-11-07,2024,NaN,BAHA Akeem,EPSI LILLE,NaN,NaN,NaN,6 mois - Manque contrat / Long terme - Manque ...
2207,None,,None,,,Demander à l'école,Actif occupé hors alternance,Analyste Modern workplace Engineering,CDI,100,...,2025-09-30,2024-11-07,2024,NaN,KOKOU BA Galeb,EPSI PARIS,NaN,NaN,NaN,6 mois - Manque contrat / Long terme - Manque ...


In [49]:
# Renommage des colonnes pour une meilleure lisibilité
colonnes = {

        "Date de la décision d'attribution de la certification" : "Date de la décision d'attribution de la certification",
        "Année d'obtention de la certification" : "Année d'obtention de la certification",
        "Le cas échéant option choisie" : "Le cas échéant option choisie",
        "Nom et Prénom du titulaire" : "Nom et Prénom du titulaire",
        "Dénomination de l'organisme ayant assuré la formation (sauf candidat libre et reconnaissance des acquis)" : "Dénomination de l'organisme ayant assuré la formation (sauf candidat libre et reconnaissance des acquis)",
        "Modalités pédagogiques (présentiel, distanciel, hybride, AFEST)" : "Modalités pédagogiques (présentiel, distanciel, hybride, AFEST)",
        "Intitulé de la formation ayant préparé au projet ou à la certification (sauf candidat libre, reconnaissance des acquis et VAE)" : "Intitulé de la formation ayant préparé au projet ou à la certification (sauf candidat libre, reconnaissance des acquis et VAE)",
#--------------------------
        "Avant - Niveau qualification" : "Avant - Niveau de qualification de la dernière certification obtenue (sans niveau, de 2 à 8, autre, non répondant)",
        "Avant - Intitulé formation" : "Avant - Intitulé exact de la dernière certification obtenue",
        "Avant - Métier" : "Avant - Dernier métier exercé",
        "Avant - Métier duree" : "Avant - Durée  d'expérience du dernier métier exercé (en nombre d'année)",
        "Avant - Métier entreprise" : "Avant - Nom de l'entreprise (1)",
        "Avant - Situation avant la certification" : "Situation avant le parcours certifiant ou à vocation certifiante (actif occupé hors alternance, en recherche d'emploi, en formation y compris alternance, inactif, autre, non répondant)",
        "Voie d'accès (formation initiale hors altenance, formation continue hors alternance, contrat d'apprentissage, contrat de professionnalisation, reconnaissance des acquis, VAE, candidat individuel, autre, non répondant)" : "Avant - Voie d'accès (formation initiale hors altenance, formation continue hors alternance, contrat d'apprentissage, contrat de professionnalisation, reconnaissance des acquis, VAE, candidat individuel, autre, non répondant)",
#--------------------------
        "6 mois - Situation après la certification" : "6 mois - Situation après la certification (actif occupé hors alternance, en recherche d'emploi, en formation y compris alternance, inactif, autre, non répondant)",
        "6 mois - Poste" : "6 mois - Intitulé du poste occupé ou de l'activité indépendante (1)",
        "6 mois - Type de contrat" : "6 mois - Type  de contrat ou de statut (CDI, CDD, CDDU, intérim, indépendant, autre, non répondant) (1)",
        "6 mois - Quotité de travail" : "6 mois - Quotité de temps de travail lié au poste occupé ou à l'activité indépendante (en %)",
        "6 mois - Entreprise" : "6 mois - Nom de l'entreprise (1)",
        "6 mois - Statut cadre / non-cadre" : "6 mois - Statut cadre (cadre, non cadre, non répondant) (1)",
        "6 mois - Date de l'enquête dans les 6 mois" : "6 mois - Date de l'enquête dans les 6 mois",
#--------------------------
        "Long terme - Situation après la certification" : "Long terme - Situation après la certification (actif occupé hors alternance, en recherche d'emploi, en formation y compris alternance, inactif, autre, non répondant)",
        "Long terme - Poste" : "Long terme - Intitulé du poste occupé ou de l'activité indépendante (1)",
        "Long terme - Type de contrat" : "Long terme - Type  de contrat ou de statut (CDI, CDD, CDDU, intérim, indépendant, autre, non répondant) (1)",
        "Long terme - Quotité de travail" : "Long terme - Quotité de temps de travail lié au poste occupé ou à l'activité indépendante (en %)",
        "Long terme - Entreprise" : "Long terme - Nom de l'entreprise (1)",
        "Long terme - Statut cadre / non-cadre" : "Long terme - Statut cadre (cadre, non cadre, non répondant) (1)",
        "Long terme - Nombre d'années après l'obtention de la certification" : "Long terme - Nombre d'années après l'obtention de la certification (en nombre d'années civiles)",
        "Long terme - Date de l'enquête actualisée" : "Long terme - Date de l'enquête actualisée",
        "Lien profil linkedin" : "Lien profil linkedin",
        "A vérifier": "A vérifier"
}

france_competence_titre = france_competence_a_verifier[list(colonnes)].rename(columns=colonnes)

### Filtre non répondant

In [50]:
# On va exporter uniquement les scrapé et enlevé les non répondant qui seront ajouter à la main du fichier de base
france_competence_final = france_competence_titre[france_competence_titre["Long terme - Intitulé du poste occupé ou de l\'activité indépendante (1)"] != "Non répondant"]

### Sélection des lignes provenant du fichier de base pour les non répondants du scraping

In [51]:
# filtre sur les non répondants du scraping
non_repondant = france_competence_titre[france_competence_titre["Long terme - Intitulé du poste occupé ou de l\'activité indépendante (1)"] == "Non répondant"]
non_repondant["Non répondant"] = non_repondant["Nom et Prénom du titulaire"]

# Merge avec fichier de base pour filtre non répondants
fichier_base = EPSI_promotion_brut
fichier_base_filtre_non_repondant = fichier_base.merge(non_repondant, how="left", left_on="Nom et Prénom du titulaire", right_on="Non répondant")

fichier_base_filtre_non_repondant = fichier_base_filtre_non_repondant[~fichier_base_filtre_non_repondant["Non répondant"].isna()]

C:\Users\aurel\AppData\Local\Temp\ipykernel_9784\3326636733.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_repondant["Non répondant"] = non_repondant["Nom et Prénom du titulaire"]


## Mise en forme et export excel

In [52]:
france_competence_final.to_excel("Tableau de promotion final V6.xlsx", index=False)

# fichier_base_filtre_non_repondant.to_excel("Tableau de promotion non répondant.xlsx", index=False) a rajouter à la main

# export manquant à scrap avec un lien linkedin

In [47]:
export_manque =  EPSI_scraping_result

# Filtre les data_brut pour garder que les colonnes utiles 

export_manque = export_manque[
    (export_manque["Lien profil linkedin"] != "") & 
    (export_manque["Lien profil linkedin"].notna()) &
    ((~export_manque["Long terme - Poste"].notna()) & (~export_manque["École 1"].notna()))
].reset_index(drop=True)

# Transformation colonne Lien profil linkedin en liste
export_manque["Lien profil linkedin"] = export_manque["Lien profil linkedin"].apply(lambda x: [x] if pd.notnull(x) else [])

# Merge tout dans une seule liste
all_profile_urls = [url for sublist in export_manque["Lien profil linkedin"] for url in sublist]

# 2) Sauvegarder la liste dans un fichier JSON
with open("profil_a_scrap.json", "w", encoding="utf-8") as f:
    json.dump(all_profile_urls, f, indent=4, ensure_ascii=False)